In [1]:
import itertools
import numpy as np
import operator 
import stanza
import spacy
from nltk.corpus import stopwords
import nltk
import csv
import string
import pandas as pd
from datetime import datetime
nltk.download('stopwords')
import matplotlib.pyplot as plt
from operator import is_not
from functools import partial
import networkx as nx
from ast import literal_eval
from nltk.tokenize import MWETokenizer
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ספיר\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url = 'data/'
# Read the xlsx file of high-tech job ads
df_excelFile = pd.read_excel(url+'Israeli high-tech job ads.xls')

In [3]:
df_excelFile.head()

,Job title,Sub title,Company name,Company description,Role description,Job requirements,Job scope,Location,min experience,max experience,Date publication
0,Server Side QA Engineer,Quality Assurance,888Holdings,"When you think online gaming, it’s hard not to...",We are looking for a Server QA Engineer with s...,At least 2 years experience in manual testing....,Full time,"Herzliyya, Tel-Aviv",3,undefined,8.8.2020
1,Fullstack Developer,"Software engineer, programmer",888Holdings,"When you think online gaming, it’s hard not to...",is looking for a highly motivated and brillia...,3+ years of experience and Strong web fundamen...,Full time,Herzliya and Ramat-Hasharon,5,undefined,8.9.2020
2,Solution Architect,"System Architect, Software Engineer",888Holdings,888 is a global online gaming company with man...,We are seeking a driven and innovative solutio...,"Bachelor’s degree in information technology, c...",Full time,Herzliya and Ramat Hasharon,6,undefined,24.9.2020
3,Data QA Engineer,Quality Assurance,888holdings,"888 group, one of the world’s most popular onl...",is looking for a highly motivated Data QA Engi...,3+ years of experience as a QA data engineer.\...,Full time,Herzliya,3,undefined,8.8.2020
4,Data Science Manager- Freelance - Temporary Po...,"Computers and software, Data science",888Holdings,"888 group, one of the world's most popular onl...",is looking for a part-time Freelance Data Scie...,MSc / PhD in a quantitative field (Computer Sc...,Part time,Herzliya and Ramat Hasharon,3,undefined,10.10.2020


In [4]:
# Enter window size (integer between 1 and n) and number of jumps (integer between 1 and n-1)
W = int(input ('Please Select window size W: '))
S = int(input ('Please Select step size S: '))

Please Select window size W: 3
Please Select step size S: 2


# **Creating a data set**




In [5]:
def sort_date(df_excelFile):
    '''
    A function that sorts dataframe by date.
    '''
    df_excelFile['Date  publication']=pd.DatetimeIndex(pd.to_datetime(df_excelFile['Date  publication'], format = "%d.%m.%Y")).date
    df_excelFile.sort_values(by=['Date  publication'], inplace=True)
    return df_excelFile

In [6]:
df_excelFile=sort_date(df_excelFile)
df_excelFile.head()

,Job title,Sub title,Company name,Company description,Role description,Job requirements,Job scope,Location,min experience,max experience,Date publication
973,Mobile UA Manager,"Marketing, Internet",Plarium\n,Plarium was founded in 2009. In 2017 it was ac...,Our goal is to optimize our campaigns with the...,1+ year digital marketing experience.\nUser ac...,Full time,Herzliya,1,2,2020-01-03
985,BI Team Leader,BI,Playtika\n\n,Playtika BI is responsible for processing more...,We are looking for a BI Team Leader to join us...,Bachelor degree – Must (preferably Computer Sc...,Full time,Herzliya,3,4,2020-01-06
722,Senior Software Engineer,Software Development,K Health,K Health is transforming the world of medicine...,We are looking for a senior software engineer ...,Proven software engineering experience.\nInteg...,Full time,Tel Aviv Yafo,3,4,2020-01-08
754,Social Media Account Manager - Replacement for...,Infrastructure,LivePerson \n\n,LivePerson is a transformational force in how ...,LivePerson is looking for a Social Media Accou...,2-4 years of hands on experience with managing...,Full time,Raanana,3,4,2020-01-09
1381,Full-Stack Team Leader,Computer Software,YouAppi,YouAppi is a leading performance-based mobile ...,YouAppi is looking for Hands On Fullstack Team...,At least two years of experience as a full sta...,Full time,Raanana,2,3,2020-01-11


In [7]:
def sort_ads_by_date(df_excelFile):
    '''
    sort ads by date function
    A function that receives an ads dataframe.
    The function sorts ads by data => Select window size W
    and steps size S => Group ads while moving window.
    '''
    df_sort_ads_date, index= pd.DataFrame(), 0
    for i in df_excelFile.iloc:
        # Create a single W-size list
        row=[tuple(df_excelFile.iloc[j][ind] for ind in range(11)) for j in range(index,W+index) if(j<len(df_excelFile))]
        # Create a nested list
        if len(row)==W:
            df_sort_ads_date = df_sort_ads_date.append([row], ignore_index=True)
        index += S
    return df_sort_ads_date

In [8]:
df_sort_ads_date=sort_ads_by_date(df_excelFile)
df_sort_ads_date.head(10)

,0,1,2
0,"(Mobile UA Manager, Marketing, Internet, Plari...","(BI Team Leader, BI, Playtika\n\n, Playtika BI...","(Senior Software Engineer, Software Developmen..."
1,"(Senior Software Engineer, Software Developmen...",(Social Media Account Manager - Replacement fo...,"(Full-Stack Team Leader, Computer Software, Yo..."
2,"(Full-Stack Team Leader, Computer Software, Yo...","(Senior Data Scientist, Data science, Machine ...","(Senior Graphics Engine Engineer, Software Dev..."
3,"(Senior Graphics Engine Engineer, Software Dev...","(Embedded Software Engineer, Engineering, Acti...","(Forward Deployed Software Engineer, Engineeri..."
4,"(Forward Deployed Software Engineer, Engineeri...","(Machine Learning Engineer, Machine learning, ...",(Team Lead Support Engineer – Machine Learning...
5,(Team Lead Support Engineer – Machine Learning...,"(R&D Team Leader, Engineering, Confidential Co...","(Computer Vision Algorithms Engineer, Computer..."
6,"(Computer Vision Algorithms Engineer, Computer...","(Data Engineer Team leader, Data Engineer, Dat...","(BackEnd Software Engineer, Engineering, Info..."
7,"(BackEnd Software Engineer, Engineering, Info...","(Field Application Engineer (Part of R&D), Sof...","(Frontend Developer, Information Technology, D..."
8,"(Frontend Developer, Information Technology, D...","(Senior Deep-learning Applied Researcher, Comp...","(Devops - CI/CD Team Lead, Information Service..."
9,"(Devops - CI/CD Team Lead, Information Service...","(Full Stack Team Leader, Marketing & Advertisi...","(Experienced Java Developer, Software Developm..."


In [9]:
df_sort_ads_date.to_csv(url+'sort ads date.csv',index=False,header=False)

# Part 2


In [10]:
def column_merging(df_sort_ads_date):
    '''
    For each data column, build a join text from this column in window.
    The function gets a dataframe named df_sort_ads_date and returns a list of merged columns.
    '''
    zip_columns=pd.DataFrame()
    for document in range(0, len(df_sort_ads_date)):
      zip_column=[list(zip(*df_sort_ads_date.iloc[document]))]
      zip_columns=zip_columns.append(list(zip_column), ignore_index=True)
    return zip_columns

In [11]:
zip_columns= column_merging(df_sort_ads_date)

In [12]:
zip_columns.to_csv(url+'merge columns.csv',index=False,header=False)

In [13]:
pd.DataFrame(zip_columns).head()

,0,1,2,3,4,5,6,7,8,9,10
0,"(Mobile UA Manager, BI Team Leader, Senior Sof...","(Marketing, Internet, BI, Software Development)","(Plarium\n, Playtika\n\n, K Health)",(Plarium was founded in 2009. In 2017 it was a...,(Our goal is to optimize our campaigns with th...,(1+ year digital marketing experience.\nUser a...,"(Full time, Full time, Full time)","(Herzliya, Herzliya, Tel Aviv Yafo)","(1, 3, 3)","(2, 4, 4)","(2020-01-03, 2020-01-06, 2020-01-08)"
1,"(Senior Software Engineer, Social Media Accoun...","(Software Development, Infrastructure, Compute...","(K Health, LivePerson \n\n, YouAppi)",(K Health is transforming the world of medicin...,(We are looking for a senior software engineer...,(Proven software engineering experience.\nInte...,"(Full time, Full time, Full time)","(Tel Aviv Yafo, Raanana, Raanana)","(3, 3, 2)","(4, 4, 3)","(2020-01-08, 2020-01-09, 2020-01-11)"
2,"(Full-Stack Team Leader, Senior Data Scientist...","(Computer Software, Data science, Machine lear...","(YouAppi, Soluto, PLAYSTUDIOS\n)",(YouAppi is a leading performance-based mobile...,(YouAppi is looking for Hands On Fullstack Tea...,(At least two years of experience as a full st...,"(Full time, Full time, Full time)","(Raanana, Tel Aviv (Sarona) , Tel Aviv Yafo)","(2, 6, 5)","(3, undefined, undefined)","(2020-01-11, 2020-01-11, 2020-01-14)"
3,"(Senior Graphics Engine Engineer, Embedded Sof...","(Software Development, Engineering, Engineerin...","(PLAYSTUDIOS\n, Action-Item , Palantir Technol...",(Playstudios develops multiplayer real-time mo...,"(We are seeking a talented, motivated, and exp...",(A minimum of 7 years’ relevant work experienc...,"(Full time, Full time, Full time)","(Tel Aviv Yafo, Tel Aviv, Tel Aviv)","(5, 0, 1)","(undefined, undefined, 2)","(2020-01-14, 2020-02-08, 2020-02-12)"
4,"(Forward Deployed Software Engineer, Machine L...","(Engineering, Information Technology, Machine ...","(Palantir Technologies, cnvrg.io, cnvrg.io)","(At Palantir, we’re passionate about building ...","(You work with customers around the globe, whe...","(Strong engineering background, preferred in f...","(Full time, Full time, Full time)","(Tel Aviv, Tel Aviv, Jerusalem)","(1, 4, 2)","(2, 5, undefined)","(2020-02-12, 2020-03-29, 2020-03-31)"


In [14]:
def preprocessing_input(zip_columns):
    '''
    A function that converts tuple data to a string without a date column to obtain a suitable input for preprocessing
    '''
    data_to_preprocessing=pd.DataFrame()
    for document in range(0, len(zip_columns)):
      add_separator_list=list([' '.join(zip_columns.iloc[document][sentence]) for sentence in range(0,8)])
      data_to_preprocessing = data_to_preprocessing.append([add_separator_list], ignore_index=True)
    return data_to_preprocessing

In [15]:
data_to_preprocessing=preprocessing_input(zip_columns)
data_to_preprocessing.head()

,0,1,2,3,4,5,6,7
0,Mobile UA Manager BI Team Leader Senior Softwa...,"Marketing, Internet BI Software Development",Plarium\n Playtika\n\n K Health,Plarium was founded in 2009. In 2017 it was ac...,Our goal is to optimize our campaigns with the...,1+ year digital marketing experience.\nUser ac...,Full time Full time Full time,Herzliya Herzliya Tel Aviv Yafo
1,Senior Software Engineer Social Media Account ...,Software Development Infrastructure Computer S...,K Health LivePerson \n\n YouAppi,K Health is transforming the world of medicine...,We are looking for a senior software engineer ...,Proven software engineering experience.\nInteg...,Full time Full time Full time,Tel Aviv Yafo Raanana Raanana
2,Full-Stack Team Leader Senior Data Scientist S...,"Computer Software Data science, Machine learni...",YouAppi Soluto PLAYSTUDIOS\n,YouAppi is a leading performance-based mobile ...,YouAppi is looking for Hands On Fullstack Team...,At least two years of experience as a full sta...,Full time Full time Full time,Raanana Tel Aviv (Sarona) Tel Aviv Yafo
3,Senior Graphics Engine Engineer Embedded Softw...,"Software Development Engineering Engineering, ...",PLAYSTUDIOS\n Action-Item Palantir Technologies,Playstudios develops multiplayer real-time mob...,"We are seeking a talented, motivated, and expe...",A minimum of 7 years’ relevant work experience...,Full time Full time Full time,Tel Aviv Yafo Tel Aviv Tel Aviv
4,Forward Deployed Software Engineer Machine Lea...,"Engineering, Information Technology Machine le...",Palantir Technologies cnvrg.io cnvrg.io,"At Palantir, we’re passionate about building s...","You work with customers around the globe, wher...","Strong engineering background, preferred in fi...",Full time Full time Full time,Tel Aviv Tel Aviv Jerusalem


# **`Preprocessing`**

In [16]:
def remove_unicode_characters(df):
    '''
    In order to avoid a small number of character combinations that stanza could not decompose, we will perform replace and encode
    The function removes\u202f,\u200b,\ufeff,\u200e,\u2005,\u2004,\u2006,\u2008 and returns an updated data frame
    '''
    for i in range(len(df.iloc[0])):
        df[i]=df[i].str.encode('utf-8').str.decode('ascii', 'ignore').str.replace('/',' / ').str.replace('\\',' \\ ').str.replace('-',' - ').str.replace('~',' ~ ').str.replace(')',' ) ').str.replace('(',' ( ').str.replace('+',' + ')
    return df

In [17]:
data_to_preprocessing= remove_unicode_characters(data_to_preprocessing)

In [18]:
data_to_preprocessing.head()

,0,1,2,3,4,5,6,7
0,Mobile UA Manager BI Team Leader Senior Softwa...,"Marketing, Internet BI Software Development",Plarium\n Playtika\n\n K Health,Plarium was founded in 2009. In 2017 it was ac...,Our goal is to optimize our campaigns with the...,1 + year digital marketing experience.\nUser ...,Full time Full time Full time,Herzliya Herzliya Tel Aviv Yafo
1,Senior Software Engineer Social Media Account ...,Software Development Infrastructure Computer S...,K Health LivePerson \n\n YouAppi,K Health is transforming the world of medicine...,We are looking for a senior software engineer ...,Proven software engineering experience.\nInteg...,Full time Full time Full time,Tel Aviv Yafo Raanana Raanana
2,Full - Stack Team Leader Senior Data Scientist...,"Computer Software Data science, Machine learni...",YouAppi Soluto PLAYSTUDIOS\n,YouAppi is a leading performance - based mobil...,YouAppi is looking for Hands On Fullstack Team...,At least two years of experience as a full sta...,Full time Full time Full time,Raanana Tel Aviv ( Sarona ) Tel Aviv Yafo
3,Senior Graphics Engine Engineer Embedded Softw...,"Software Development Engineering Engineering, ...",PLAYSTUDIOS\n Action - Item Palantir Technolo...,Playstudios develops multiplayer real - time m...,"We are seeking a talented, motivated, and expe...",A minimum of 7 years relevant work experience....,Full time Full time Full time,Tel Aviv Yafo Tel Aviv Tel Aviv
4,Forward Deployed Software Engineer Machine Lea...,"Engineering, Information Technology Machine le...",Palantir Technologies cnvrg.io cnvrg.io,"At Palantir, were passionate about building so...","You work with customers around the globe, wher...","Strong engineering background, preferred in fi...",Full time Full time Full time,Tel Aviv Tel Aviv Jerusalem


In [19]:
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('en', processors={'tokenize':'spacy'}, use_gpu=True) # This sets up a default neural pipeline in English

2021-06-24 00:01:52 INFO: Downloading default packages for language: en (English)...
2021-06-24 00:01:54 INFO: File exists: C:\Users\ספיר\stanza_resources\en\default.zip.
2021-06-24 00:01:58 INFO: Finished downloading models and saved to C:\Users\ספיר\stanza_resources.
2021-06-24 00:01:58 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | spacy     |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-06-24 00:01:58 INFO: Use device: cpu
2021-06-24 00:01:58 INFO: Loading: tokenize
2021-06-24 00:01:58 INFO: Loading: pos
2021-06-24 00:02:00 INFO: Loading: lemma
2021-06-24 00:02:00 INFO: Loading: depparse
2021-06-24 00:02:01 INFO: Loading: sentiment
2021-06-24 00:02:02 INFO: Loading: ner
2021-06-24 00:02:03 INFO: Done loading processors!


In [20]:
def lower_and_tokens_data(df_preprocessing):
    '''
    A function that receives a data frame, divides tokens and converts the data into lowercase letters.
    The function returns an updated data frame.
    '''
    # Decompose text into tokens and convert to lowercase
    df_lower_tokens_for_documents=pd.DataFrame()
    #len(data_to_preprocessing)
    for document in range(0,2):
        lower_tokens_for_document=[]
        for sentence_ in range(0,len(df_preprocessing.iloc[0])):
            doc=nlp(df_preprocessing.values.tolist()[document][sentence_])
            # Create a list of tokens lists
            lower_tokens_for_document.append([[token.text.lower() for token in sentence.tokens] for i,sentence in enumerate(doc.sentences)])
        df_lower_tokens_for_documents = df_lower_tokens_for_documents.append([[list(itertools.chain.from_iterable(sentence)) for sentence in lower_tokens_for_document]],ignore_index=True) 
    return df_lower_tokens_for_documents

In [21]:
df_lower_tokens_for_documents=lower_and_tokens_data(data_to_preprocessing)

In [36]:
df_lower_tokens_for_documents.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team, leader, senior...","[marketing, ,, internet, bi, software, develop...","[plarium, \n , playtika, \n\n , k, health]","[plarium, was, founded, in, 2009, ., in, 2017,...","[our, goal, is, to, optimize, our, campaigns, ...","[1, +, , year, digital, marketing, experience...","[full, time, full, time, full, time]","[herzliya, herzliya, tel, aviv, yafo]"
1,"[senior, software, engineer, social, media, ac...","[software, development, infrastructure, comput...","[k, health, liveperson, \n\n , youappi]","[k, health, is, transforming, the, world, of, ...","[we, are, looking, for, a, senior, software, e...","[proven, software, engineering, experience, .,...","[full, time, full, time, full, time]","[tel, aviv, yafo, raanana, raanana]"
2,"[full, -, stack, team, leader, senior, data, s...","[computer, software, data, science, ,, machine...","[youappi, soluto, playstudios, \n]","[youappi, is, a, leading, performance, -, base...","[youappi, is, looking, for, hands, on, fullsta...","[at, least, two, years, of, experience, as, a,...","[full, time, full, time, full, time]","[raanana, tel, aviv, , (, sarona, ), , tel,..."
3,"[senior, graphics, engine, engineer, embedded,...","[software, development, engineering, engineeri...","[playstudios, \n , action, -, item, , palanti...","[playstudios, develops, multiplayer, real, -, ...","[we, are, seeking, a, talented, ,, motivated, ...","[a, minimum, of, 7, years, relevant, work, exp...","[full, time, full, time, full, time]","[tel, aviv, yafo, tel, aviv, tel, aviv]"
4,"[forward, deployed, software, engineer, machin...","[engineering, ,, information, technology, mach...","[palantir, technologies, cnvrg.io, cnvrg.io]","[at, palantir, ,, were, passionate, about, bui...","[you, work, with, customers, around, the, glob...","[strong, engineering, background, ,, preferred...","[full, time, full, time, full, time]","[tel, aviv, tel, aviv, jerusalem]"


In [37]:
df_lower_tokens_for_documents.to_csv(url+'lower_tokens.csv', index=False,header=False)

In [40]:
df_tokens = pd.read_csv(url+'lower_tokens.csv',header=None)

In [41]:
df_tokens.head()

,0,1,2,3,4,5,6,7
0,"['mobile', 'ua', 'manager', 'bi', 'team', 'lea...","['marketing', ',', 'internet', 'bi', 'software...","['plarium', '\n ', 'playtika', '\n\n ', 'k', '...","['plarium', 'was', 'founded', 'in', '2009', '....","['our', 'goal', 'is', 'to', 'optimize', 'our',...","['1', '+', ' ', 'year', 'digital', 'marketing'...","['full', 'time', 'full', 'time', 'full', 'time']","['herzliya', 'herzliya', 'tel', 'aviv', 'yafo']"
1,"['senior', 'software', 'engineer', 'social', '...","['software', 'development', 'infrastructure', ...","['k', 'health', 'liveperson', '\n\n ', 'youappi']","['k', 'health', 'is', 'transforming', 'the', '...","['we', 'are', 'looking', 'for', 'a', 'senior',...","['proven', 'software', 'engineering', 'experie...","['full', 'time', 'full', 'time', 'full', 'time']","['tel', 'aviv', 'yafo', 'raanana', 'raanana']"
2,"['full', '-', 'stack', 'team', 'leader', 'seni...","['computer', 'software', 'data', 'science', ',...","['youappi', 'soluto', 'playstudios', '\n']","['youappi', 'is', 'a', 'leading', 'performance...","['youappi', 'is', 'looking', 'for', 'hands', '...","['at', 'least', 'two', 'years', 'of', 'experie...","['full', 'time', 'full', 'time', 'full', 'time']","['raanana', 'tel', 'aviv', ' ', '(', 'sarona',..."
3,"['senior', 'graphics', 'engine', 'engineer', '...","['software', 'development', 'engineering', 'en...","['playstudios', '\n ', 'action', '-', 'item', ...","['playstudios', 'develops', 'multiplayer', 're...","['we', 'are', 'seeking', 'a', 'talented', ',',...","['a', 'minimum', 'of', '7', 'years', 'relevant...","['full', 'time', 'full', 'time', 'full', 'time']","['tel', 'aviv', 'yafo', 'tel', 'aviv', 'tel', ..."
4,"['forward', 'deployed', 'software', 'engineer'...","['engineering', ',', 'information', 'technolog...","['palantir', 'technologies', 'cnvrg.io', 'cnvr...","['at', 'palantir', ',', 'were', 'passionate', ...","['you', 'work', 'with', 'customers', 'around',...","['strong', 'engineering', 'background', ',', '...","['full', 'time', 'full', 'time', 'full', 'time']","['tel', 'aviv', 'tel', 'aviv', 'jerusalem']"


In [42]:
def turn_string_to_list(df_strings):
    '''
    repair file - turn string to list
    '''
    df_temp=pd.DataFrame() 
    for i in range(df_strings.shape[0]):
        df_temp = df_temp.append(df_strings.iloc[i,:].apply(lambda x: literal_eval(x)))
    return df_temp

In [43]:
df_tokens=turn_string_to_list(df_tokens)
df_tokens.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team, leader, senior...","[marketing, ,, internet, bi, software, develop...","[plarium, \n , playtika, \n\n , k, health]","[plarium, was, founded, in, 2009, ., in, 2017,...","[our, goal, is, to, optimize, our, campaigns, ...","[1, +, , year, digital, marketing, experience...","[full, time, full, time, full, time]","[herzliya, herzliya, tel, aviv, yafo]"
1,"[senior, software, engineer, social, media, ac...","[software, development, infrastructure, comput...","[k, health, liveperson, \n\n , youappi]","[k, health, is, transforming, the, world, of, ...","[we, are, looking, for, a, senior, software, e...","[proven, software, engineering, experience, .,...","[full, time, full, time, full, time]","[tel, aviv, yafo, raanana, raanana]"
2,"[full, -, stack, team, leader, senior, data, s...","[computer, software, data, science, ,, machine...","[youappi, soluto, playstudios, \n]","[youappi, is, a, leading, performance, -, base...","[youappi, is, looking, for, hands, on, fullsta...","[at, least, two, years, of, experience, as, a,...","[full, time, full, time, full, time]","[raanana, tel, aviv, , (, sarona, ), , tel,..."
3,"[senior, graphics, engine, engineer, embedded,...","[software, development, engineering, engineeri...","[playstudios, \n , action, -, item, , palanti...","[playstudios, develops, multiplayer, real, -, ...","[we, are, seeking, a, talented, ,, motivated, ...","[a, minimum, of, 7, years, relevant, work, exp...","[full, time, full, time, full, time]","[tel, aviv, yafo, tel, aviv, tel, aviv]"
4,"[forward, deployed, software, engineer, machin...","[engineering, ,, information, technology, mach...","[palantir, technologies, cnvrg.io, cnvrg.io]","[at, palantir, ,, were, passionate, about, bui...","[you, work, with, customers, around, the, glob...","[strong, engineering, background, ,, preferred...","[full, time, full, time, full, time]","[tel, aviv, tel, aviv, jerusalem]"


In [44]:
def remove_space_and_newline(df_tokens):
    '''
    A function that receives a dataframe of tokens.
    The function returns a dataframe without spaces and a newline character.
    '''
    df_remove_space = pd.DataFrame()
    for document in df_tokens.iloc:
      df_remove_space=df_remove_space.append([[[elem for elem in sentence if elem.strip()]  for sentence in document]],ignore_index=True)
    return df_remove_space

In [45]:
df_remove_space= remove_space_and_newline(df_tokens)
df_remove_space.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team, leader, senior...","[marketing, ,, internet, bi, software, develop...","[plarium, playtika, k, health]","[plarium, was, founded, in, 2009, ., in, 2017,...","[our, goal, is, to, optimize, our, campaigns, ...","[1, +, year, digital, marketing, experience, ....","[full, time, full, time, full, time]","[herzliya, herzliya, tel, aviv, yafo]"
1,"[senior, software, engineer, social, media, ac...","[software, development, infrastructure, comput...","[k, health, liveperson, youappi]","[k, health, is, transforming, the, world, of, ...","[we, are, looking, for, a, senior, software, e...","[proven, software, engineering, experience, .,...","[full, time, full, time, full, time]","[tel, aviv, yafo, raanana, raanana]"
2,"[full, -, stack, team, leader, senior, data, s...","[computer, software, data, science, ,, machine...","[youappi, soluto, playstudios]","[youappi, is, a, leading, performance, -, base...","[youappi, is, looking, for, hands, on, fullsta...","[at, least, two, years, of, experience, as, a,...","[full, time, full, time, full, time]","[raanana, tel, aviv, (, sarona, ), tel, aviv, ..."
3,"[senior, graphics, engine, engineer, embedded,...","[software, development, engineering, engineeri...","[playstudios, action, -, item, palantir, techn...","[playstudios, develops, multiplayer, real, -, ...","[we, are, seeking, a, talented, ,, motivated, ...","[a, minimum, of, 7, years, relevant, work, exp...","[full, time, full, time, full, time]","[tel, aviv, yafo, tel, aviv, tel, aviv]"
4,"[forward, deployed, software, engineer, machin...","[engineering, ,, information, technology, mach...","[palantir, technologies, cnvrg.io, cnvrg.io]","[at, palantir, ,, were, passionate, about, bui...","[you, work, with, customers, around, the, glob...","[strong, engineering, background, ,, preferred...","[full, time, full, time, full, time]","[tel, aviv, tel, aviv, jerusalem]"


In [46]:
def create_multiple_words(tuples_molti_words,df):
    '''
    A function that receives a tuple of tuples that contains words for merging and a dataframe that contains tokens.
    performs merging of multiple words.
    '''
    tokenizer = MWETokenizer()
    for molti_word in tuples_molti_words: 
      tokenizer.add_mwe(molti_word)
    return pd.DataFrame([tokenizer.tokenize(sent) for sent in docs ] for docs in df.iloc)


In [47]:
def exchanging_words_representative(word_dic,df):
    '''
    Expressions of the same meaning are replaced by a predefined representative.
    The function receives a dictionary and a dataframe.
    Exchanges the values in the dataframe, any kay value in the dictionary in the dataframe will be replaced with a value.
    '''
    for index in range(len(df.iloc[0])):
      df[index] = df[index].apply(lambda sentence: [word_dic.get(value, value) for value in sentence])
    return df


In [48]:
# Create a taple of multiple words
tuples_molti_words=(('c', '#'),('c','+','+'))
df_after_molti_words=create_multiple_words(tuples_molti_words,df_remove_space)
df_after_molti_words.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team, leader, senior...","[marketing, ,, internet, bi, software, develop...","[plarium, playtika, k, health]","[plarium, was, founded, in, 2009, ., in, 2017,...","[our, goal, is, to, optimize, our, campaigns, ...","[1, +, year, digital, marketing, experience, ....","[full, time, full, time, full, time]","[herzliya, herzliya, tel, aviv, yafo]"
1,"[senior, software, engineer, social, media, ac...","[software, development, infrastructure, comput...","[k, health, liveperson, youappi]","[k, health, is, transforming, the, world, of, ...","[we, are, looking, for, a, senior, software, e...","[proven, software, engineering, experience, .,...","[full, time, full, time, full, time]","[tel, aviv, yafo, raanana, raanana]"
2,"[full, -, stack, team, leader, senior, data, s...","[computer, software, data, science, ,, machine...","[youappi, soluto, playstudios]","[youappi, is, a, leading, performance, -, base...","[youappi, is, looking, for, hands, on, fullsta...","[at, least, two, years, of, experience, as, a,...","[full, time, full, time, full, time]","[raanana, tel, aviv, (, sarona, ), tel, aviv, ..."
3,"[senior, graphics, engine, engineer, embedded,...","[software, development, engineering, engineeri...","[playstudios, action, -, item, palantir, techn...","[playstudios, develops, multiplayer, real, -, ...","[we, are, seeking, a, talented, ,, motivated, ...","[a, minimum, of, 7, years, relevant, work, exp...","[full, time, full, time, full, time]","[tel, aviv, yafo, tel, aviv, tel, aviv]"
4,"[forward, deployed, software, engineer, machin...","[engineering, ,, information, technology, mach...","[palantir, technologies, cnvrg.io, cnvrg.io]","[at, palantir, ,, were, passionate, about, bui...","[you, work, with, customers, around, the, glob...","[strong, engineering, background, ,, preferred...","[full, time, full, time, full, time]","[tel, aviv, tel, aviv, jerusalem]"


In [49]:
# Expressions of the same meaning were replaced by a predefined representative.
word_dic = {'c_+_+':'c++','c_#':'c#'}
df_after_molti_words=exchanging_words_representative(word_dic,df_after_molti_words)
df_after_molti_words.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team, leader, senior...","[marketing, ,, internet, bi, software, develop...","[plarium, playtika, k, health]","[plarium, was, founded, in, 2009, ., in, 2017,...","[our, goal, is, to, optimize, our, campaigns, ...","[1, +, year, digital, marketing, experience, ....","[full, time, full, time, full, time]","[herzliya, herzliya, tel, aviv, yafo]"
1,"[senior, software, engineer, social, media, ac...","[software, development, infrastructure, comput...","[k, health, liveperson, youappi]","[k, health, is, transforming, the, world, of, ...","[we, are, looking, for, a, senior, software, e...","[proven, software, engineering, experience, .,...","[full, time, full, time, full, time]","[tel, aviv, yafo, raanana, raanana]"
2,"[full, -, stack, team, leader, senior, data, s...","[computer, software, data, science, ,, machine...","[youappi, soluto, playstudios]","[youappi, is, a, leading, performance, -, base...","[youappi, is, looking, for, hands, on, fullsta...","[at, least, two, years, of, experience, as, a,...","[full, time, full, time, full, time]","[raanana, tel, aviv, (, sarona, ), tel, aviv, ..."
3,"[senior, graphics, engine, engineer, embedded,...","[software, development, engineering, engineeri...","[playstudios, action, -, item, palantir, techn...","[playstudios, develops, multiplayer, real, -, ...","[we, are, seeking, a, talented, ,, motivated, ...","[a, minimum, of, 7, years, relevant, work, exp...","[full, time, full, time, full, time]","[tel, aviv, yafo, tel, aviv, tel, aviv]"
4,"[forward, deployed, software, engineer, machin...","[engineering, ,, information, technology, mach...","[palantir, technologies, cnvrg.io, cnvrg.io]","[at, palantir, ,, were, passionate, about, bui...","[you, work, with, customers, around, the, glob...","[strong, engineering, background, ,, preferred...","[full, time, full, time, full, time]","[tel, aviv, tel, aviv, jerusalem]"


In [50]:
def remove_special_characters(df_after_molti_words):
    '''
    Remove special characters function
    A function that receives a dataframe and removes special characters from it.
    '''
    # list of special characters 
    punctuations = list(string.punctuation)
    punctuations.extend(['–','’','’s',' / ','--','·','®','©',' ',' ’s','—',' -','”','...','’re','n’t',"'s","’re","'re",'ng',"'ll",'  ','s.',"n't",'‘','“','..','’ll',"we're","'ve",'i','ii','iii','iv',' \\ ','`ll','.a','us','er','wo','nt','ae', "'d",'ut'])
   
    # Remove chars that are composed of special characters 
    df_remove_special_characters=pd.DataFrame() 
    for document in df_after_molti_words.iloc:
      df_remove_special_characters=df_remove_special_characters.append([[[token for token in sentence if token not in punctuations] for sentence in document]],ignore_index=True)
    return df_remove_special_characters


In [51]:
df_remove_special_characters=remove_special_characters(df_after_molti_words)
df_remove_special_characters.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team, leader, senior...","[marketing, internet, bi, software, development]","[plarium, playtika, k, health]","[plarium, was, founded, in, 2009, in, 2017, it...","[our, goal, is, to, optimize, our, campaigns, ...","[1, year, digital, marketing, experience, user...","[full, time, full, time, full, time]","[herzliya, herzliya, tel, aviv, yafo]"
1,"[senior, software, engineer, social, media, ac...","[software, development, infrastructure, comput...","[k, health, liveperson, youappi]","[k, health, is, transforming, the, world, of, ...","[we, are, looking, for, a, senior, software, e...","[proven, software, engineering, experience, in...","[full, time, full, time, full, time]","[tel, aviv, yafo, raanana, raanana]"
2,"[full, stack, team, leader, senior, data, scie...","[computer, software, data, science, machine, l...","[youappi, soluto, playstudios]","[youappi, is, a, leading, performance, based, ...","[youappi, is, looking, for, hands, on, fullsta...","[at, least, two, years, of, experience, as, a,...","[full, time, full, time, full, time]","[raanana, tel, aviv, sarona, tel, aviv, yafo]"
3,"[senior, graphics, engine, engineer, embedded,...","[software, development, engineering, engineeri...","[playstudios, action, item, palantir, technolo...","[playstudios, develops, multiplayer, real, tim...","[we, are, seeking, a, talented, motivated, and...","[a, minimum, of, 7, years, relevant, work, exp...","[full, time, full, time, full, time]","[tel, aviv, yafo, tel, aviv, tel, aviv]"
4,"[forward, deployed, software, engineer, machin...","[engineering, information, technology, machine...","[palantir, technologies, cnvrg.io, cnvrg.io]","[at, palantir, were, passionate, about, buildi...","[you, work, with, customers, around, the, glob...","[strong, engineering, background, preferred, i...","[full, time, full, time, full, time]","[tel, aviv, tel, aviv, jerusalem]"


In [52]:
def remove_stop_words(df_remove_special_characters):
    '''
    A function that receives a dataframe and removes stop words
    '''
    df_filtered_stop_words=pd.DataFrame()
    stop = stopwords.words('english')
    for document in df_remove_special_characters.iloc:
      df_filtered_stop_words=df_filtered_stop_words.append([[[token for token in sentence if token not in stop] for sentence in document]],ignore_index=True)
    return df_filtered_stop_words


In [53]:
df_filtered_stop_words=remove_stop_words(df_remove_special_characters)
df_filtered_stop_words.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team, leader, senior...","[marketing, internet, bi, software, development]","[plarium, playtika, k, health]","[plarium, founded, 2009, 2017, acquired, arist...","[goal, optimize, campaigns, effective, combina...","[1, year, digital, marketing, experience, user...","[full, time, full, time, full, time]","[herzliya, herzliya, tel, aviv, yafo]"
1,"[senior, software, engineer, social, media, ac...","[software, development, infrastructure, comput...","[k, health, liveperson, youappi]","[k, health, transforming, world, medicine, kno...","[looking, senior, software, engineer, design, ...","[proven, software, engineering, experience, in...","[full, time, full, time, full, time]","[tel, aviv, yafo, raanana, raanana]"
2,"[full, stack, team, leader, senior, data, scie...","[computer, software, data, science, machine, l...","[youappi, soluto, playstudios]","[youappi, leading, performance, based, mobile,...","[youappi, looking, hands, fullstack, team, lea...","[least, two, years, experience, full, stack, t...","[full, time, full, time, full, time]","[raanana, tel, aviv, sarona, tel, aviv, yafo]"
3,"[senior, graphics, engine, engineer, embedded,...","[software, development, engineering, engineeri...","[playstudios, action, item, palantir, technolo...","[playstudios, develops, multiplayer, real, tim...","[seeking, talented, motivated, experienced, se...","[minimum, 7, years, relevant, work, experience...","[full, time, full, time, full, time]","[tel, aviv, yafo, tel, aviv, tel, aviv]"
4,"[forward, deployed, software, engineer, machin...","[engineering, information, technology, machine...","[palantir, technologies, cnvrg.io, cnvrg.io]","[palantir, passionate, building, software, sol...","[work, customers, around, globe, gain, rare, i...","[strong, engineering, background, preferred, f...","[full, time, full, time, full, time]","[tel, aviv, tel, aviv, jerusalem]"


In [54]:
# Create a taple of multiple words

                    # Terms of the database
tuples_molti_words_all_data=(('deep', 'learning'),('2', '3','years'),('global','e'),('e', 'start', 'system'),('deep','machine','learning'),('2', '4','network'),( '2','g','3','g','4','g', '5','g'),('deep', 'subject'),('c++','14', '17','20'),('c++','11','17'), ('deep','reinforcement','learning'),('data', 'engineer'),('software', 'engineer'),('software','engineering'),('team', 'leader'),('team', 'lead'),('software', 'development'),('full', 'stack'),('new','cost','effective'),
                    ('data','scientist'), ('new','startup'),('new','image'),('new',  'web'),( 'deep','subject','matter','expert'),( 'new','programming','languages'),('machine', 'learning') ,('data', 'science'),('quality', 'assurance'),('computer', 'vision'),('full', 'stack'),('start','up'),('start','ups'),('covid','19','ct'),
                    ('.net','framework', '4.x'),('dedicated','intelligence','research'),('true','team','player'),(  'true', 'gamers'),( 'true','professionals'),
                    ( 'saving', 'work'),('work', 'locations'),( 'part','team'),  ( 'work','migration'),( 'work', 'manage'),('work', 'experience','3','years'),
                    ('computer','software'),('information','technology'),('big','data'),('high', 'tech', 'solutions'),('high', 'tech'),( 'neural', 'networks'),('multi','disciplinary'),('5','g', 'nr'),
                    ('back', 'end'),('react', 'developer'),('react', 'native'),('product', 'management'),('mechanical', 'design'),('new','end','end'),('automation', 'engineer'),('a','l'),('cyber', 'security'),
                    ('front','end'),('team', 'player'),('server','side'),('lorry', 'i.', 'lokey'),('manual', 'qa' ),('large','scale'),('qa','team'),('scrum','methodologies'),('deployment','processes'),('test','engineering'),
                    ('test','engineer'), ('process', 'engineer'),('mrb','processes'),('multi','tasking'),('windows', 'server'),('multi','functional'),('networks','cloud'),('cloud','based'),('new','ci','cd'),
                    ('quality', 'manager'),('manual', 'testing'),('manual', 'tests'),('test','plans'),('testing','plan'),('3d','printing'),( 'hands','on'),('3d','world'),('4','g', '5','g', 'nr'),
                    ('medical', 'devices'),('scientific','discipline'),('medical', 'device'),('strategic', 'quality'),('technology','providers'),('manufacturing', 'ecosystem'),('integrated', 'circuits'),
                    ('problem','solvers'),('medical', 'research'),('life', 'cycle'),('pro','actively'),('high','profile'),('cross','functional'),('information','technology'),   
                    ('sprint', 'planning'),('program', 'manager'),('qa', 'automation'),('high','performance'),('team', 'members'),( 'eight', 'countries'),( 'eight', 'decades'),
                    ('non','technical'),('data','analysis'),('ideal','candidate'),('data','scientists'),('public', 'relations'),('web','applications'),('network', 'security'),('wi','fi'),('dev','ops'),
                    ('agile','development'),('cutting','edge'),('technical', 'service'),('product', 'analyst'),('product', 'analytics'),('high', 'level'),('product','led'), ('full','motion'), 
                    ( '2,000','month'),('2,000','global','customers'),('gpt', '2'),('2,000','customers'),('least', '2', 'following', 'expertise'),('2','teams'),('entry','level'),  
                    ('application', 'security'),('bio','medical','engineering'), ('automation', 'lead'), ('data','mining'),('mlops','methodologies'),('deployment','process'),('cloud','technologies'),
                    ('definition',   'ab',   'tests'),
                    ('model', 'training'), ('service','oriented'),('customer','orientation'), ('landing','pages'),('new','open','source'),('behavioral','analytics'),('model', 'development' ),('development','algorithms'),
                    ('analytical', 'solutions'),('fraud','ring', 'attacks'),( 'data','structures'),('code', 'review'),('open', 'source'),('research', 'teams'),('research' ,'engineers'),('world','class'),
                    ('visual', 'design'),('text', 'classification'),('3', 'networking'),('3','4','developers'),('sentiment', 'analysis'),('time','series'),('graph', 'theory'),('online','experimentation'),('client','side'),
                    ('custom','made'),('off','line'),('system', 'support'),('non','conformities'),('reinforcement', 'learning'),('test', 'hypotheses'),('validate', 'models'),('can','do'),('systems','cloud'),
                    ('independently', 'learn'),('validation','plan'),('dynamic','analysis'),('reverse','engineering'),('cloud','native'),('electronics', 'engineering' ),('computer', 'science'),
                    ('wireless', 'communications'),('scheduling','methods'),('algorithm', 'team'),('data', 'preparation'),('imaging', 'systems'),('data', 'analytics'),('electro','optical'),
                    ('2d','image', 'processing'),('image', 'processing'),('sensor', 'processing'),('over', 'the', 'wall'),('data','driven', 'ecommerce'),('autonomous', 'vehicles'),('public','facing'),
                    ('computer', 'hardware'),('algorithm', 'developer'),('new','levels'),('signal', 'processing'),( 'linux','based'),('innovation', 'team'),('industry','standard'),('ar','field'),
                    ('ar','world'),('open','minded'),('business', 'analytics'),('natural', 'language', 'processing'),('global', 'economy'),('nlp', 'research'),('scikit','learn'),('financial', 'services'),
                    ('learning','based'),('multi','camera'),('image', 'pipeline'),('ab',   'tests'),('database', 'design'),('data', 'collection'),('storage', 'techniques'),('data','quality'),('consumer', 'electronics'),
                    ('real','time'),('data', 'pipeline'),('collecting', 'data'),('security', 'teams'),('mobile', 'team'),('health', 'care'),('re','usable'),('programmed', 'clean'),('strategy','building'),
                    ('system','design'),('chip','embedded'),('work','fast','paced'),('fast', 'technologies'),('technical','field'),('manage','team'),('eda','tools'),('emea', 'area'),('application', 'engineering'),
                    ('sd','wan'),('fast','pacing'),('micro','services'),('super','innovative'),('proxy', 'tools'),('technology', 'division'),('end','end'),('scrum', 'team'),('automation','developer'),
                    ('code', 'ok'),('aws','stack'),('business','driven'),('payments', 'network'),('web' 'builder'),('quality', 'standards'),('infrastructure', 'developer'),('artificial','intelligence'),
                    ('data', 'protection'),('data','governance'),('testing', 'methodologies'),('integration','focused'),('system','wide'),('multi','threaded'),('language', 'developer'),('make', 'pro'),
                    ('functional', 'language'),('non','functional'),('mobile', 'platforms'),('data','base'),('highly','available'),('spring', 'boot'),('spring', 'data'),('web','api'),
                    ('java', 'script'),('ui','testing'),('data','platform'),('fast','growing'),('scale','data'),('data','intensive'),('intensive','systems'),('azure', 'cloud'), ('azure', 'cloud'),
                    ( 'ai','algorithms'),('qa','engineer'),('java', 'multithreading'),('full', 'effect'),('multiple', 'sclerosis'),('up','to','date'),('higher','converting'),('planning','api'),
                    ('innovating', 'technologies'),('frontend', 'frameworks'),('off','the','shelf'),('can','do','attitude'),('embedded', 'software'),('cost','effective'),('verification', 'test'),
                    ('emulation', 'techniques'),('emulator','environment'),('system', 'verilog'),('physical', 'design'),('test','writing'),('sign','off'),('dv', 'engineer'),('dv', 'methodology'),
                    ('tape','out'),('bug','free'),('pre','silicon'),('security', 'research', 'team'),('penetration', 'testing'),('elastic', 'block', 'store'),('fault','tolerant'),('object','oriented'),
                    ('context', 'api'),('react','hooks'),('satellite', 'services'),('booking','holdings'),('product','videos'),('production', 'engineering'),('product','engineering'),('product','engineer'),
                    ('support', 'engineers'),('quality','engineering'),('dedicated', 'intelligence', 'research'),('certificate','analysis'),('technical', 'data', 'sheets'),('define','product'),
                    ('software','development','lifecycle'),('production','level'),('program', 'management', 'office'),('director','construction'),('project', 'management', 'professional'),
                    ('fab', 'construction', 'enterprise'), ('co','founders'),('data','set'),('sub','systems'),('pre','processing'),('floor','planning'),('timing', 'analysis'),
                    ('logic','optimization'),('risc','v'),('facilities','contracts', 'manager' ),('per', 'need'),('manufacturing', 'product', 'engineering'),('frequencies', 'measurements'),
                    ('test', 'methods'),('multi','task'),('team','players'),('technical_lead'),( 'manufactures', 'systems'),('data', 'sets'),('video', 'games'),
                    ('optimization', 'processes'),('landing', 'pages'),('website', 'traffic'),('ad','hoc'),('paid','campaigns'),('google', 'analytics'),('a','b', 'testing'),('mobility','service'), 
                    ('network','appliances'),('engineering','process'),('linux', 'system' ),('robotics','experts'),('security','nitro'),('ibm', 'qradar'),('cisco', 'ids'),('palo', 'alto'),
                    ('microsoft', 'windows'),('digital', 'touchpoint'),('digital','marketing' ),('metrics','driven'),('user', 'acquisition'),('mobile', 'ecosystem'),
                    ('curious', 'mind'),('quick', 'learner'),('high','self','learning'),('high','degree','organization'),('acquisition', 'channels'),('marketing','campaigns'),('user','acquisition', 'teams'),
                    ('key', 'performance'),('major','metrics'),('roi','goals'),('achieve', 'volume'),('structured','data'),('data','modeling'),('data','lake'),('lead','team'),
                    ('analysis', 'tools'),('reporting','tools'),('design','dwh'),('analytics','solutions'),('playtika', 'games'),('ambiguous', 'challenges'),('executing', 'sophisticated'),
                    ('leaders','plan'),('tough', 'problems'),('creative', 'ways'),('risk', 'management'),('co','innovation'),('cross','functionally'),('co','design'),('well','funded'),
                    ('fast','growing'),('digital','native'),('next','generation'),('subject','matter', 'expert'),('people','oriented'),('cyber','attacks'),('program','management','professional'),
                    ('top','tier'),('spot','im'),('simple','use'),('cyber','attack'),('super','tech'),('soc','stakeholders'),('long','lasting'),('fundamental','data'),('e','financing'),
                    ('recommendation','systems'),('3d','vector'),('nist','sp','800','115'),('rest','api'),('endpoint','technologies'),('gen','v' ),('owasp','top','10'),('sans','25'),
                    ('iec','62443'),('java','11'),('spring','boot','2'),('75', 'million'),('top','nlp'),('e','commerce'),('ever','growing'),('new','features', 'months'),('900','employees'),
                    ('leave', '9', 'months'),('six','months','year'),('6','months','ago'),( '8', 'months'),('2','million','users'),('us','army'),('around','7', 'months'),('identifying', 'bugs'),
                    ('iso','14001'),('iso','45001'),('iso9001','2015'),('iso', '14971'),('iso', '13485'),('iso','13485:2016'),('iso','27001'),('iso','9001'),('fda','21'),('series', 'b'),
                    ('cfr','part', '820'),('iso','62304'),('iso','27001','14971'),('pci','dss'),('iso','20000'),('8200', 'graduate'),('360','investigations'),('100','billion'),
                    ('rio','360'),('bmi','360'),('edge', '360'),('js','3'),('css','2'),('176','million'),('100,000', 'organizations'),('10','it','technical'),
                    ('oversee', '100'),('100','hands'),('online','marketing', 'platform'),('established','2010'),('100,000', 'experts'),('established','2011'),('established','1997'),
                    ('videocites','established'),('several','serial','entrepreneurs'),('2007','established','secretary','department'),('established','1992'),('electrical', 'engineering'),
                    ('100','employees'),('founded','2012'),('refinery', '29'),('founded','2009'),('more','700','top','tier'),('over', '100m','led'),('1.8','billion'),('over','7500', 'people'),
                    ('37','countries'),( 'over','856m','2019'),('25','disciplines'),('set','up','1976'),('acquired','2019'),('3rd','party'),('286','million'),('200','markets'),('56','currencies'),
                    ('25','currencies'),('fintech','company'),('18','high','growth'),('200000','merchants'),('nearly','100', 'years'),('1500','employees'),('820','million'),('100','employees'),
                    ('since','2010'),('8','year','old'),('26','million'),( '500','companies'),('since', '80', 'breaches'),('100','countries'),('10000','enterprise'),('.net','core', '2.x','3.x'),('boostrap','3.x'),
                    ('7400', 'enterprise'),('146','million'),('10','best','startups'),('april', '2020'),('2,018','offers'),('100','mobile'),('50','startups','2019'),('israel','2018'),
                    ('over','500','employees'),('10','most','dseirable'),('founded','2010'),('37.8','million'),('30','engineers'),('10','most','impactful'),('blockchain','2019','2020'),
                    ('over','10','billion'),('general','catalyst'),('method', 'capital'),('thomvest', 'ventures'),('translink', 'capital'),('telstra','ventures'),('10','company'),
                    ('fully','remote'),('500 000','applications'),('gpu','1999'),('red', 'bynet', 'group'),('over','2500','computer'),('system','availability','99.99'),('simply','888'),
                    ('180', 'microservices'),('1.5b','revenue'),('1.5','million', 'developers'),('70','frontend'),('30','backend'),('more','300','years','combined','experience'),
                    ('37.8','million','date'),('300','540','games'),('over','910','million','users'),('may','2020'),('3,400','cities'),('across','112','countries'),('45','languages'),
                    ('data','loss','prevention'),('test','generation'),('2','7','ngrx','store'),('ionic','3'),( '5,800','customers'),( 'least','2','senior','level'),('4,000','professionals'),
                    ('over','700000','local','editors' ),('mooviters', 'account'),('77','hundreds','cities'),('3d','2d'),('1st', 'contact'),('150','people'),('success','rate','almost','100'),
                    ('888','established', 'more','20','years'),('888','s','tech','teams'),('established','early','2016'),('tier','1', 'customers'),('covid','19'),('manufacturing','product','engineering'),
                    ('safer','drive','24','7'),('vehicle','systems'),('deep', 'neural', 'networks'),('100s','ways'),('gig','platforms'),('fintech', 'area'),('aws','preferred'),
                    ('bayesian', 'optimization'),('highly','organized'),('fluent','english'),('strong','written'),('develop','features','80'),('co','founder'),('28','total','energy'),
                    ('building','truly', 'unique', 'technology'),('test', 'new', 'ideas'),('angular','2','9'),('1.2', 'billion'),('over', '1,200','employees'),('3','5','engineers'),
                    ('old','infrastructure'),('new','technologies'),('new','products'),('new','product'),('new','markets'),('new','industry'),('new','team','members'),('public','cloud'),
                    ('new','feature'),('new','optimized'),('new','services'),('new','paradigm'),('new','capabilities'),('new','team'),('new','paas','serve'),('new','methods'),
                    ('new','software','products'),('new','tools'),('new','things'),('new','ways'),('new','sensor'),('new','disciplines'),('new','capabilities'),('888','holdings'),
                    ('new','wireless'),('new','groundbreaking'),('new','medical'),('new','innovative'),('new','business'),('new','creative'),('new','modules'),('new','areas'),('138','million'),
                    ('new','current'),('new','market'),('new','subjects'),('new','propietery'),('new','emulation'),('new','chip'),('new','system'),('new','designs'),('join','new','israel','office'),
                    ('new','unmanaged'),('new','attacks'),('new','exploits'),('new', 'opportunities'),('from','1b','6b'),('angular', 'cli'),('45,000','customers'),('21','cfr'),
                    ('new','cutting'),('new','perspective'),('new','solutions'),('new','automation'),('new','languages'),('new','employee'),('new','software'),('new','threat'),
                    ('new','way','organizations'),('more','secure','future'),('new','orleans','pelicans'),('israeli','security', 'agency'),('working','cloud'),('leadership','skills'),
                    ('big','picture'),('big', 'advantage'),('iec','60601'),('iatf16949','2016'),('big','parts'),('more','60,000'),('licenses', 'deployed'),('full','spectrum'),('serve', '80'),
                    ('hybrid','environment'),('hybrid','storage'),('hybrid','cloud'),('hybrid','soc','architecture'),('hybrid', 'platforms'),('ip','protection','hybrid'),('hybrid','systems'),
                    ('hybrid','environments'),('hybrid','networks'),('unique','hybrid'),('hybrid','full','working'),('over', '130','countries'),('100','software','based'),('initially', 'used','2017'),
                    ('founded','2017'),('2017','acquired','aristocrat'),('over','450','million','shoppers'),('spports','over','450','local'),('more','450','billion','times'),('over','26','million','users'),
                    ('offices','26','countries'),('30','50','systems'),('70','big','data'),('os','x'),('terabytes','data'),('threat','hunters'),('python','3.0'),('go','1.13'),('nearly','2.3','billion','people'),
                    ('2018','forbes','cloud'),('100','list'),('company','started','2011'),('established','2011'),('founded','2011'),('started','2011'),('12','offices'),('data','qa'),('di', 'frameworks'),
                    ('lorry','i.','lokey','nterdisciplinary'),('series','e'),('ex','ifa'),('ip','enabled'),('2.6','billion'),('september','2020'),('24','7','noc'),('365','working'),
                    ('7', 'layer','model'),('70','hands'),('nn', 'compression'),('29','locations'),('strong','office', '365'),('office', '365'),('integrate', 'dynamics','365'),('6,200','customers'),
                    ('part','dynamic'),('build','contribute','innovative'),('build','company','product'),('build','maintain', 'scalable'),('team', 'building'),('over','62.5m'),
                    ('build','internal', 'tools'),('build','automated', 'deployments'),('building','robust'),('build','test','automation'),('nearly','40','years'),('40','people'),
                    ('skills','english'),('solid', 'innovation', 'skills'),('cutting','edge','skills'),( 'people', 'skills'),('good','communication', 'skills'),
                    ('engineering','skills'),('written','communication','skills'),('written','skills','english'),('verbal','communication','skills'),('strong', 'interpersonal','skills'),
                    ('over','20,000','enterprises'),('over', '500,000','applications'),('more','25','years'),('since', '1988'),('over', '18,000','customers'),('more','250','csp'),
                    ('world','wide'),('over','1,300','experts'),('1,300', 'employees'),('more','1,300','employees'),('2nd','3rd','year'),('2nd','generation'),('more','40', 'billion'),
                    ('3','5','developers'),('3','5','members'),('tier','1','publications'),('tier','1'),('tier','one'),('ex', 'libris'),('decision','making'),('temporary','position'),
                    ('over','70','programmers'),('delivered','above','70'),('10','100','tb'),('more','70','billion'),('over','70','countries'),('over','70','programmers'),
                    ('by','2030'),('40','million','ev'),('join','team'),('join','company'),('join','founding', 'team'),('join','effort','validating'),('join','software','team'),
                    ('join','challenging'),('methodologies.p','inc.'),('join','npi','team'),('join','development'),('join','group'),('join','analytics','department'),('join','family'),
                    ('tech','lead'),('join','journey'),('join','growing','data','science'),('c++','11'),('c++','14'),('11','14', 'development'),('5','g','group'),('5','g' ,'massive'),
                    ('20','years','ago'),('one','world'),('least', '4', 'develop'),('angular','2'),('least', '3','them'),('.net','4.5'),('40','50','people'),('50','time'),('fortune', '500'),
                    ('2000','deployments'),('2000','it','environments'),('average', '92'),('3d','reconstruction'),('direct','focus','3d'),('3d','rendering'),('3d','graphics'),
                    ('3d','computer', 'vision'),('3d', 'data'),('4d','3d', 'applications'),('3d','background'),('3d','mapping'),('3d','electrodes'),('3d', 'computer'),('3d', 'reconstruction'),
                    ('3d','video'),('advance', '3d'),('3d','platform'),('3d', 'image'),('3d','2d'),('2d','3d'),('gpa', '85'),('average','85'),('average','mark', '85'),('25','countries'),
                    ('past','25','years'),('saves', 'clients', '25'),('18','25','generation'),('25','disciplines'),('25m', 'conversations'),('sae', '21434'),('6','continents'),('since','2004'),
                    ('beverage','market','1903'),('100','countries','worldwide'),('fortune', '100', 'companies'),('100', 'countries'),('over','100','developers'),('100','countries'),
                    ('104','cities'),('104', 'million', 'users'),('24','million'),('active', 'diners'),('available', '24', 'hours'),('24', 'hours'),('ecma','script','6'),( 'technology', 'fast', '500'),
                    ('more than 100 A/B tests'),('100', 'company'),('up','240'),('240','people'),('founded','1985'),('over','450,000','developers'),('manage', '450,000', 'emergency'),
                    ('over', '240,000,000','people'),('dr.','noah', 'liel'),('michal','yaacovi'),('2019','fastest','growing'),('2019','facilitate','rescue'),('launched','2019'),('new','deep','learning'),
                    ('startup','work','2019'),('over','856m','2019'),('acquired','2019'),('100','top','companies','2019'),('microsoft', '2008','2019'),('es2019', 'development'),('deep', 'learning'),
                    ('blockchain', 'industry','2019','2020'),('q4','2019', 'alone'),('11','international','markets'),('over','11','consecutive','years'),('11', 'countries'),('14','software','developer'),
                    ('38','languages'),('o.o', 'concepts'),('least','300','messages','per','second'),('300','employees'),('300','trucks'),('300','payment', 'methods'),('more','300', 'years'),
                    ('300','540','games'),('over','300','communication', 'service','providers'),('more', '300','support', 'associates'),('120','languages'),('more','120', 'networking'),('18','cities','worldwide'),
                    ('18','high','growth'),('10','12','18','months'),('over','3,000','employees'),('more','3,000', 'employees'),('about','3,000','employees'),('operate','2,000','content'),('rich', 'websites'),
                    ('john', 'wanamaker','1838','1922'),('170','team'),('1700','employees'),('over','170','countries'),('over', '170','team', 'members'),('founded','1994'),('sap','started','1972'),
                    ('112','countries'),('global 2000'),('more','2000','deployments'),('25,000','employees'),('Over','75','time'),('more','75','million'),('including','75','fortune', '100'),
                    ('over','1,500','products'),('50','dosage','forms'),('about','1,500','employees'),('more','370','locations'),('10,000,000','people'),('over','110,000','registered'),
                    ('gpu', '84'),('help','12,000','brands'),('over','175','clients'),('over', '175','million'),('ipc','610'),('ipc','620'),('ipc','600'),('do','160'),('40,000','thermal','imaging'),
                    ('do','178'),('do','254cqe'),('more','1,800', 'payment','specialists'),('over', '1,800','leading','organizations'),('over','1,600','workers'),('95','local','currencies'),
                    ('supporting','95','intel','revenue'),('gpa','least','95.0','100'),('founded','back','1983'),('over','800','other','games'),('more','800', 'employees'),('800','million'),
                    ( '2006','amazon','web'),('since', 'establishment', '2006'),('already', '110,000', 'registered'),('365', 'attract'),('established', '2014'),('founded','2014'),('2020', 'list'),
                    ('2013','heart','raleigh'),('founded','2013'),('since', 'launch','april','2013'),('400','categories'),('across', '400','enterprises'),( 'new','develop'),('network','2,000,000'),
                    ('80', 'nationalities'),('28','nm'),( '280', 'million','sessions'),( 'founded','2002'),('established','2002'),('4d','images'),('server', 'application'),('application','depth'),
                    ( '138','million'),('240,000,000','people'),('1200','customer'),('175','partners'),('2000','environment'),('2000','turn'),( 'set', '1976'),('32','bit'),('iso','27032'),('iso', '27017'),
                    ('around','27','years'),('early', '1997'),('700', 'top_tier'),('team','7'),('beer', 'sheba'),('develop','24',  '7'),('10,337','employees'),('founded','2007'),('since','2007'),
                    ('founded', '2015'),('20','fortune'),('250','models'),('20','largest'),('20','car'),('250','models'),( 'v.','kubernetes'),('23','countries'),('cyberattacks','2030'),('place', '2030'),
                    ('2030', '40','million','ev'),( 'develop', 'team'),('15', 'employees'),('3', 'linux'),( 'raised ','146','venture'),('team', '3', '4'),('2','g'),('3','g'),('4','g'),('5','g'),
                    ('4','territories'),('40','million','homeowners'),('40','billion'),('55,000','professionals'),('october', '2020'),('455,000', 'customer'),( '130', 'employees'),
                    ('2.7','million'),('8','million'),('2.5', 'billion'),('since','2016'),('since', '2019'),('30', 'awards'),('3', 'digiday'), ('1', 'digiday'),('16','telly', 'awards'),('10','muse','creative','awards'),
                    ('2','webby', 'awards'),('2','lovie','awards'),('100','k'),('2016', 'shaker','acquired'),( '2016', 'shaker'),( '18,000', 'customers'),('1,200',  'people'),('100','k'),
                    ('92','forbes','global'),( '100', 'earned','recognition'),('2009','company','development','center'),('including','90', 'top'),('100', 'banks', 'globally'),('ranging', '1b','6b','annual'),
                    ('minimum', '4','semesters'),('manning', '24','7',  'technical'),( 'receive', 'money','100','currencies'),('investors',  '30','years'),('2', '3', 'pc'),('8','spring'),('certified', 'university','85'),
                    ('1','billion'),('165','million'),('founded','2005'),( 'january', '21st','2020'),('450,000','developers'),('c','11'),('c', '14', '17', '20'),('angular','6'),( '10,000', 'enterprise'),( 'manage', '3','4','remote'),
                    ('3,500','people'),('40','countries'),('10,000','companies'),( '92', 'transportation'),('3.000', 'travel','agencies'),( 'funding', '2','million'),( '2015',  'series'),('funding', '20', 'million'),
                    ( '80',  'million',  '2018'),('2016', 'funding','injection'),('august',  '2020'),( '15', '20', 'million'),('founded', 'paris', '2012'),( 'since', 'opened', 'offices'),('700', 'enterprises'),
                    ('supports', '450','local'),('4',  'workforce'),('2007','grown','become', 'leading'),('300','communication','service'),( '300,000', 'merchants'),('300,000', 'financial'),('300,000','restaurants'),('300', 'years'),
                    ('integrating', '300',  'security'),('225,000','restaurants'),('4,000','u.s.','cities'),('135','partners','directors'),('1,200','employees'),('1,200','customers'),('june',  '2014'),('sw','team'),('sw','developer'),
                    ('acquirer', '2015'),('Jenkins','2.0'),('launched', 'epower','2.0'),('84','please'),('founded','1999'),('first','implemented','1999'),('experts', '2013',  'vision'),('2013', 'funding'),('9','billion'),
                    ('9,000', 'customers'),( 'team', '50', 'talented'),( '2016', 'raised'),('146',  'venture'),('pioneer',   '2019'),('recognized','2019'),('next','generation', '911'),('angular','9'),('system','availability','99.99'),
                    ('20,000',  'enterprises'),('300','engineers'),('growing','30'),('300','support'),('16',  'support'),('800','games'),('8000', 'customers','worldwide'),('1,800','payment'),('1,500','employees'),('2,500','computer'),
                    ('500','tech',  'employees'),( '500',  'enterprises'),('50', 'countries'),('160','countries'),( '910',  'million','users'),('introduced', '2012'),( '91', 'million',  'users'), ('196', 'different', 'countries'),
                    ('founded','1989'),( '250', 'million'),('registered','users'),('srver','2008','2012'),('raised', '112,500',  'employees'),('3.0','jenkins'),('least','3','full','stack'),('30','50', 'faster'),
                    ('quota',  'attainment', '20', '100'),('raised','100', 'emergence'),('3', 'day','week'),('3','developers'),('3','engineers'),('1,5','billion'),('raised', '62.5',  'investors'),('february','2020'),
                    ('commit', 'work', '4', 'days'),('3','5','writing','samples'),('3','days','per',  'week'),('work', '3', 'shifts',  'week'),('1', 'rated','job'),( 'new', 'factories'),('new','scope'),('new','features'),
                    ('manual', 'test'),('cases', 'new', 'features'),('oo','design','patterns'),('developing','oo'),('oo','languages'),('oo','javascript'),('e','learning'),('new','technology'),('new','challenges'),
                    ('new','medicines'),('new',  'trick',  'iphone'),('new','tactics'),('new','platforms'),('w','engineering'),('new', 'autonomous','technologies'),('80','countries'),( 'providing', '1,500', 'products'),
                    ('300,000','mission'),('least', '3', 'semesters', 'graduation'),('3,000','employees'),('350','experts'),('team', '30'),('30,000','merchants'),( '7', 'billion', 'mobile'),('130,000',  'apps'),('3', 'best', 'platforms'),
                    ('one', 'top'),('360','group'),('3','semesters'),('least', '3','days', 'week'),('360','project'),('team','3','6','people'),( 'least', '3','team','members'),( '35', 'index'),('founded','2003'),
                    ('370', 'locations'),('least','3',  'java'),('3,500', 'products'),('dealing','30'),('3.5','billion'), ('3','senior'),('build','3', 'different', 'devops'),('google','play','1'),( '2019', 'apple'),('3', 'times','app'),
                    ('25','conversations', 'globality'),('companies','2019'),('1','users'),( 'soc', 'level', '3'),( '250', 'csp', 'customers'), ('130', 'countries'),( 'report', '36'),('python', '2.7','3'),( 'top', '3', 'sites'), 
                    ('3','experience'),('boostrap', '3.x', 'font'),('60','employees','worldwide'), ('37', 'israel'),('100','customers'),('two','rounds'),('30','million'),('3','billion'),('30','k'),('150','tb'),                   
                    ('3', 'backend'),('50', 'percent'),('30', 'percent'),('37','million'),('30,000','organizations'),('deep', 'experience'),( '1.5', 'billion'),('inversion','control'),( 'pro', 'elementor'),
                    ('360', 'view'),( '3', 'hardware'),('high', 'performing'),('teams', '3'),('3',  'browser'),('7000','customers'),( 'networking','layers','3','5'),('close', '200', 'enterprise'),('7,500','people'),
                    ('gross',  'profit', '856', '2019'),('four', 'core'),('lead', '2',   '4', 'product'),('working', 'pro'),('active', 'attitude'),('mfg', 'pro'),('pro', 'activeness'),('new', 'way'),('new','data'),
                    ( 'windows', '7'),('pro','windows',  '8.1'),('pro','windows','10'),('pro', 'knowledge','mac'),('pro', 'active'),( 'ida','pro'),('os', 'x.'),('knowledge' 'basic'),('f', 'external'),('ideas','z.'),
                    ('200','destinations'),('await','ioc.'),('integrated', 'dynamics', '365'),('190','countries'),('2019','invested'),('15', 'sales'),('q3', '2019'),('500','customers'),
                    ( 'invest','24'),('launched','operations', '1996'),('startup',  '2019', 'prestigious'),( 'company', 'started', '1988'),('founded', '1981'),('established','1975'),('around','350', 'employees'),
                    ('build',  '2019',  'momentum'),( 'includes','65','patents'),('198', 'trademark'),('4.1','billion', 'fiscal', '2019'),('ensure', 'product', 'quality', '70'),('launched', '2017'),( 'angular','7'),
                    ( '700,000', 'local', 'editors'),('l7', 'networking', 'kubernetes'),('170','countries'),('24', '7','production'),('70', 'backend'),('30',  'frontend'),( '70',  'front','side'),('700', 'top','tier'),
                    ('new','material'),('new', 'insights'),( 'new','flows'),('new', 'stuff'),('learn', 'new','stuff'),(  'set', 'new','smart'),('new', 'knowledge'),('new', 'ai','system'),('new',  'sensors'),
                    ('new','security'), ('new', 'senior') ,('new','games'),('new', 'customers'),('new', 'user'),('high','product','quality'),('new','heights'),('new','campaigns'),('new','machine','learning'),
                    ('new','country'),('new','amazing', 'data'),('new','ideas'),('new','european'),('new','exciting','project'),('new', 'micro','services'),('new','level'),('new','people'),('creating','new','connections'),
                    ('new','effective','ways'),('new', 'construction', 'data'),('vendor','2019', 'insurance', 'autodesk'),('already','110,000','registered','users'),('150','million'),('8500', 'customers'),
                    ( 'january','21', '2020'),('12', 'factor', 'experience'),( '600','million','users'),('15','products'),( '2018', 'mobileye',  'revealed'),('8','connect'),( '1,800','leading', 'organizations'),
                    ('50', 'utilized','algosecs'),('450,000','developers'),('10,000','enterprise'),('130,000','apps'),('510','k'),('new', 'documents'),( 'well', 'hands'),('2018','hurricane', 'florence'),
                    ('founded','2016'),('150,000', 'people'),('180','locations'),('3,400', 'cities'),('1', 'urban'),('100','developers'),('100,000', 'companies'),('100s', 'millions'),( 'old','900','employees'),
                    ('fall','2018'), ('since'  'launched', '15', 'cities'),( '200,000',  'businesses'),('100s','thousands'),('1.5','developers'),('new','relic'),( 'team', '100', 'members'),
                    ('2018','offers'),('50','marketing'),('180','countries'),('50','languages'),('1', '000', '000','unique'),('c','11', '17'),( 'regular', '1'),('1', 'meetings'),('1,300','experts'),
                    ('15','million'),('500,000', 'unique'),('1',  'video'),('10', 'office', '2016'),('1','worldwide'),('founded','2016'), ('180',  'locations'),('raised', '240'),
                    ( '55,000',  'professionals'),('100','million','users'),('forbes', '20'),('1,500', 'products'),('1,600', 'workers'),('1,200','employees'),('100','million'),
                    ('1,000', 'government'),('1,300','experts'),( 'micro',  'controllers', '8',  '16'),('1,300','employees'),('dream', '2025'),  ('since', 'foundation', '2001'),('80', 'mergers'),
                    ( 'b2b','mobility'),('services','2010'),('100','leading'),('advanced','technology'),( '150,000', 'employees'),('45','countries'),( '9.7','billion'),( 'every', '10', 'seconds'),
                    ( 'june',  '4',  '2020'),('16,000',  'customers'),('100', 'led'),('100', 'professionals'),( '2', 'sided'),('gpa', '90','100'),('hands','on','sw'),('3.5','days'),                   
                    ( 'company', 'supporting', '90'),( '5,800', 'customers'),('60,000','licenses','deployed'), ('150',  'countries'),( 'past', '100', 'years'),('6,000', 'employees'),
                    ('january', '21', '2020'),('150','million'),('services', '2010'),('1', 'app'),('100,000', 'store'),( 'visual', 'studio', '2017'),('requirements','240'),('os','x'),
                    ('1','g'),('100', 'g'),( '1,800','leading', 'organizations'),( '50','utilized'),('emergency', '911'),('eco','system'),('100','professionals'),('2018','acquired'),                   
                    ('18,000', 'customers'),( '7,400', 'enterprise'),( '18,000', 'brands'),( '1', 'top'), ('20', 'companies'),('175','clients'),('tier','2'),('every', '10', 'weeks'),('200', 'websites', 'world'),
                    ( '17,000', 'enterprise'),('175','countries'),('over','70,000'),('12', 'factor'),('from','a','z'),('bold','vision'),('wont','happen'),('175', 'million'),('7', 'developers'),('17,000', 'people'),
                    ('246','million'),('8','500','enterprise'),( '40', 'tb'), ('way', 'around', 'b','tests'),('using', 'b', 'tests'),( '300','000','merchants'),('300'', ''000','financial','instruments'),('1.2','million'),
                    ('300', '000',  'restaurants'),( '300', '000', 'mission'),( '300', '000',  'restaurants'),( '225',  '000',  'restaurants'),( '4', '000','u.s.'),('44', 'international'),('250','exchanges'),('africa', '2001'), 
                    ('iso','26262'),( 'level', '4'),('managing','l4'),( 'growing', '4'),('150',  'employees'),('420', 'employees'),('45', 'million'),('ownership', '24', '7'),( 'high','scale'),
                    ('layer', '2'),('4', 'network'),('team', '4', 'devops'),('48', 'million'),( '24', 'series'),('4', 'developers'),('6,000','technology'),('6,500', 'customers'),('maintenance', '24', '7', 'continuous'),
                    ('6,000','global', 'customers'),('8', 'bit'),('16', 'bit'),('5','6', 'ms'),('ef', '6'),('angular', '4'),('since','founding', '2016'),('2010','acquired'),('sql','server', '2008', '2012'),
                    ('since',  'start', '2012'),('65', 'industry'),('600', 'million'),( '12,000','brands'),('6','000','technology'),('60', 'years',  'ago'),('6,000','global','customers'),('60,000', 'experts'),
                    ('80','team','members'),('performance',  '80', '90'),('tasks', '10',  '20'),('experience','nn'),( '800', 'employees'), ('20', 'subsidiaries'),('3', 'minutes', 'average'),('15','markets'),
                    ('6','tb'),('productivity', '6'),( '200', 'customers'),('16',  'billion'),( '30', 'billion', 'assets'), ('management', '6'),('management', '6','7'),( '36,000','employees'),
                    ('project', 'z.'),( 'b', 'forms', 'testing'),('100','languages'),('perform','b','testing'),('80','customers'),('least', 'two', 'programming',  'languages'),( '1', 'cloud','database'),('1', 'nosql'),
                    ( '40','million'), ('50', 'promising'), ('israel','2020'),(  '240,000,000', 'people'),('450', '000','emergency'),('2.5','million'),('25', 'hours', 'week'),('2.5', '3', 'days', 'week'),
                    ('4',  'days'),( 'family','250', 'strong'),('25', 'hands', 'work'),('1', '800', 'payment'), ('60','global','mobile'),('600', 'employees'),('new','revenue'),('new','users'),('2','000', 'month'),
                    ( '700', '000', 'local', 'editors'),('700', 'clients'),('level', 'serve'),('rs233','485'),('rs','485'),('60','years'),('1,900','people'),('already','600','000'),
                    ('q3','2020'),('100', 'production'),('2000',  'large',  'government'),('2000', 'employees'), ('2000','broadband'),('new','challenger'),('2000','environments'),('engineering','industry'),
                    ('4.0', 'software'),('40','talented','employees'),('4', 'engineers'),('4','million','led'),('40', '000','thermal'),('4','local'),('47','countries'),('450','billion'),
                    ('3','4','engineers'),('4','000','professionals'),('40', '50',  'plan'),('production','24','7','systems'),('dr.','yossi'),('one','largest','technology'),('6', 'million', 'downloads'),
                    ('1200','customers'), ('70', 'countries'),('5','7','engineers'), ('172','supported'),('70','programmers'),('75','time'),('5','7','employees'),('7','ngrx','store'),('html', '5'),
                    ('24',  'countries'),('5','continents'),('455,000','customers'),('development','manager', 'l4'), ('7,500','people'),('half','trillion','dollars'),('value','2025'),('python 3.x'),
                    ('250','payment'),('95', '000','online','stores'),('51','percent'),('450','million'),('19', 'less','stressed'),('14','development'),( '1', '300', 'experts'),('88', 'fortunes', 'slots'),
                    ('70','billion'),('2','4','product'),('jenkins','2.0'),('2,000','hospitals'),('10','times'),('1000','companies'),('index','100'),( 'since', '2008'),('java', '8', 'experience'),
                    ('2014','one','busiest','years'),('sql','server' ),('sql','server','2014','2016'),('least', '2','backend'), ('4,000','restaurants'), ('40','offices'), ('4,000', 'talents'),('laptops','3.x'),
                    ('90', 'different'), ('2', '4', 'raised'),('100','counting'),('5','million'),('2020', 'convizit'),( '30', 'top'),('less', '30','employees'),('founded', '2008'),('tcp','http'),
                    ('95,000',  'online','stores') ,('2,500', 'computer','partner'),('50',  'different',  'payment'), ('200',  'global'), ('30', 'worldwide', 'destinations'),('20', '30', 'time'),('b','corp'),
                    ( '30','people'),('120', 'networking'),('10', 'impactful','companies'),('80', 'hands'),('20', 'years'), ('50','professionals'),('least','20', 'hours','week'),( '6,500', 'customer'),
                    ('8,500', 'enterprise'),('robot','service'),('proxy','technologies'),('big','advantage'),('edge','devices'),('ci','cd'),('framework','4'),('ten','steps'),('leading',  '5', 'people'),
                    ( '2,500', 'computer'),('150','enterprise','custmers'),('130','million'),('80', 'supplier'),('50', 'million'),('100', 'even', 'complex'),('20', 'markets'),('three', 'top'),('font','awesome'),
                    ('50', 'eyeballs'), ('500', 'employees'),( 'windows', '10'), ('500','customer'),('20','prestigious'),('200', 'countries'), ('200', 'million'),( '2,000', 'content'),('team', '5'),('python','v3.x'),
                    ('30,000', 'merchants'),('20,000', 'enterprises'),('3,500',  'people'), ('90','countries'),('funding','round', '200','put'),('90','countries'),('30','hours'),('five',  'communication'),
                    ('500','hospitals'),('solve', '80', 'cases'),( '240,600','employees'),('200','top'),('200','accounts' ),('2', 'million'),('office', '2016'),('tel','div', '20'),('raised','112'),
                    ('4.0', 'applications'),('2,000','paying' 'industry'),('2', 'million','dollars'),('200', 'accounts'),('top', '10'),( '150', 'enterprise'),( 'ranking', 'companies'),('11', 'consecutive', 'years'),
                    ('100', 'users'),('least', '50','write'),('100,000', 'organizations'),('including', '50', 'fortune'),('500', 'companies'),('80', 'people'),('150', 'company'),('2020', 'tools'),
                    ( '30', '50','time'),('summer', '2020'), ('2,000,000', 'vacation'),('1','wire'),('10','impactful'),( '20', 'israeli','startups'),('hundreds','millions', 'customers'),('600', 'daily', 'user'),
                    ('600,000','registered','users'),('order', '4,000', 'restaurants'),('ten', 'global', 'offices'),('named','one','calcalist'),('ranked','8th'),('top','50','startups'),('8,500','enterprise'),
                    ('1', 'conversation','intelligence'),('1', 'barrier', 'adopting'),('multi', 'million', 'dollar'),('central','system'),('software', 'automation'),('design', 'systems'), ('mahwah','nj'),
                    ('http','tcp','stack'),('tcp','http','stack'),('tcp','http','networking'),('tcp','http'),('http','tcp'),('tcp','ip'),('ip','tcp'),('ip','level'),('soc','level','designs'),('full','soc','solutions'),
                    ('basic','ip','routing'),('network','technologies'),('ip','tunneling'),('ip','filtering'),('ip','conten','preparation'),('24x7','shifts'),('service','ip','protection'),
                    ('strong','ip','portfolio'),('ip', 'protection'),('lan','wan','technologies'),('eda','tools'),('snps','cdns','tools'),('technical','field'),('lan','wan'),('wan','lan'),('dns','client','side'),
                    ('temporal','reasoning'),('sensor','fusion'),('multi','device','issues'),('sh','hw','components'),('good','time'),('aws','experience'),('code','standards'),('ui','designer'),('b','testing'),('model','deploying'),
                    ('deep', 'understanding'),('ee','preferred'),('similar','field'),('back','2005'),('ny', 'dfs'),('tiers', '2', '3'),('teams', 'assisting'),('support','team'),( 'x', 'similar',  'toolings'),('3d''virtual','reality'),
                    ('multi','tier'),('multi','threading'),('multi','tasking'),('multi','object','tracking'),('multi','task'),('requested','goods'),('tech','savvy'),('scans','x', 'rays'),('x','ray'),('convolutional','neural','networks'),
                    ('high','communication','skills'),('task','management','skills'),('organizational','skills'),('high','scripting','skills'),('soft','skills'),('presenting','skills'),('qualitative','analytical','skills'),
                    ('social','skills'),('gen','z','audiences'),('owning', 'process','z'),('z','responsibility'),('top','50','startups','2019'),('2', 'mobile','applications'),( 'relevant', 'area'),
                    ('build','relationships'),( '2', 'team', 'management'),('1', '2', 'times'),('end','2','end'),('dota', '2'),('experience',  '2','following', 'programming'),( '2', 'eng','hand'),('1', '2', 'weeks'),('however','one','thing','common'),
                    ( 'per', 'trip'),('owner', 'least',  '2'),('develop', 'least',  '2'),('platforms',  '2,000'),('one',  'fastest',  'growing'),('2','yearsofexperienein'),('2', 'product'),( 'coming','soon', '2'),('one','lead','initiatives'),
                    ('least',  '2', 'following','areas'),('1,000',  'employees'),('1,000,000', 'unique', 'users'),( '15', 'sites'),('including',  'two',  'live'),( 'experiences', '85', 'countries'),('requirment', '25'),
                    ('good','understanding'),('passion','good'),('good','design'),('feel','good','factor'),('good','communication'),('very','good','interpersonal'),('good','problem'),('good','communication','skills'),('proficient', 'one'),
                    ('good','logical','thinking'),('very','good','level'),('good','interpersonal','skills'),('good','social','skills'),('good','opportunity'),('good','organizational','skills'),('virtual','logical','networking'),
                    ('very','good','english'), ('good','oral','written'),('early',  'beginnings', '2008'),('across',  '8',  'verticals'),('raised', '58'),('80','distributors'),('leading', 'team', '5', 'engineers'),('level','working'),
                    ('5',  'members'),('es6','8'),('operates', '55', 'countries'),('since', 'launched', '15',  'cities'),('become', 'one','fastest','growing'),('analysis',  '8d'),('python','3.x'),('core', '2.x',  '3.x.'),('one','single'),
                    ( '200.000','merchants'),('millions','consumer',  'leading'),('200','technologies','expert'),('150','currencies'),('investigation', '2020'),('new', 'module', 'platform'),( 'threat', 'intelligence'),('integration', 'system'),
                    ( 'continue','lead','market'),('provide','3', 'shifts','per','week'),('management','3','4','developers'),('css','3'),('vue', 'd3'),('3','million', 'users'),('knowledge', 'layer'),('virtual','reality'),
                    ( '3', 'network', 'protocols'),('l3', 'ensure', 'security'),('experience',  'cisco'),('routing', 'l3', 'l2'),('switching', 'firewalling'),( 'load', 'balancing'),('vue','js'),('non','interrupted'),('non', 'trivial'),
                    ('top','notch'),('mission','critical','product'),('high','performance'),( 'elastic','compute','cloud'),('8','connect'),( 'env0','2018'),('non',  'tech'),( 'non', 'routine'),('non','public'),
                    ('non', 'relational'),('non','exhaustive'),( 'manufacturing','data'),('non', 'invasive'),('non',  'dev', 'team'),('functional','non', 'function', 'testing'),('non','conforming',  'material'),('non', 'linear'),
                    ('non','public'),( 'non', 'financial'),('non','classic'),( 'non', 'paid'),('non', 'iid'),( 'non', 'structured'),( 'non', 'human'),('non', 'sql'),('backend','framework'),('experience','least',  'one'),
                    ('one',  'following'),( 'frameworks',  'react'),('least', 'one', 'scripting'),('programming','language'),('one', 'popular'),('least',  'one', 'large'),('one', 'large'),( 'cloud', 'provider'),(  'one', 'scientific',  'fields'),
                    ( 'really',  'great', 'people'),('k.','works'),('health','capable'),('health', 'solution'),('evidence','based'),('hr','outsourcing'),('hr','management', 'tools'),('one','resilience'),('one','never', 'forget'),
                    ('process','strategy'),('aligning','procedures'),('fulfilled', 'whatever'),('old', 'fashioned'),('low','level'),('low','rate'),('fully','operational','24x7'),( '200', 'technology', 'experts'),('one','advanced'),
                    ('working','24x7'),('maintaining','24x7'),('24x7','business'),('24x7','shifts'),('six','top'),('eight','technology','companies'),('ag','operations','z'),('one','else'),('one', 'pioneers'),
                    ('six','billion'),('anonymous','data','points'),('life', 'sciences'),('888', 'global'),('8b', 'events'),( 'one', 'chegg'),('one', 'simple'),('one','greatest'),('d.', 'experience'),('non','compliance'),
                    ('one', 'technologies'),('least', 'one'),('one','largest'),( 'one', 'first'),( 'experience',  'one',  'relational'),( 'one', 'nosql'),('least', 'one', 'main', 'language'),( 'one',  'take','pride'),('one', 'system'),
                    ('clicked',  'one', 'links'),('one','leading', 'global'),('one','leading','global'),('one', 'day'),( 'one', 'respected'),('one', 'markets', 'operate'),('one', 'healthiest'),('mastery','one'),('one', 'key'),
                    ('one', 'market'),( 'experience',  'one'),('experience', 'one',  'major'),('cloud',  'service'),( 'methodologies', 'scrum'),('one', 'standard', 'agile'),('one', 'place'),( 'global', 'partners'),('every',  'one'),
                    ('one', 'stop'),('one', 'roof'),('one', 'worlds', 'largest'),('one','programming','language'),( 'one', 'promising'),('one', 'leading', 'placement'),('one','multidisciplinary'),( 'one', 'robust','model'),('trained','data'),
                    ('trained','data'),('one','major'),( 'one', 'holistic'),('one', 'biggest'),('one',  'talent'),('one',  'components'),('one', 'design'),( 'one','glamorous'),( 'one',  'worlds'),('one',  'israels'),( 'one','cloud'),('one', 'sparkling'),
                    ('part', 'one'),('scrum',  'teams'),('one',  'leading'),( 'leader', 'one', 'teams'),('one', 'customers'),( 'high', 'growth'),( 'two', 'gaming'),('one', 'india'),('second','israel'),('one',  'acquiring'),('platform', 'globally'),
                    ( 'said', 'one'),('team', 'one'),( 'one', 'back','end'),('one','solution'),('one',  'integration'),('one', 'nt'),( 'easy', 'goals'),('one','please'),('one', 'ink', 'set'),('kinds', 'fabrics'),('one', 'set'),('every', 'day'),('forward','thinking'),
                    ('one',  'continuous'),('one', 'clouds'), ('lead', 'one', 'challenging'),( 'teams', 'work'),('using','one'),( 'one','point'),('management', 'system'),('geo','mapping'),('one','time'),('follow','through'),('work','one'),
                    ('source', 'code'),('experience', 'source', 'code'),('one','scripting'),( 'one','significant'),('one', 'hand'),('since','opened', 'doors'),('unique','users'),('one', 'billion'),('times','month'),('team','algo'),('one','elite'),
                    ( 'one','analogue'),( 'one','application'),('join', 'one', 'core'),('product', 'teams'),( 'one', 'hottest', 'work'),('one',  'founders'),('one', 'modern'),('programming','languages'),('one', 'piece'),('leading', 'logistics'),
                    ('web', 'threats'),('one', '50','promising','startups'),('one', 'cardinal'),( 'lead', 'one', 'cloudinarys'),('team','leads'),('team', 'owners'),('one','platform'),('less', 'one'),( 'cyber', 'solution'),('one','comprehensive', 'field'),
                    ( 'one', 'marketplace'),( 'one', 'areas'),('one', 'highest', 'rated'),('demand', 'one','another'),('one', 'influencing'),('join','one','success'),('casual', 'game'),('one', 'purpose'),('one','man'),('group','one','israel'),
                    ('one','ideas'),('one','year'),( 'one', 'best', 'companies'),('join','one','development', 'teams'),('one', 'b2c', 'companies'),('onto', 'one', 'saas', 'platform'),('one', 'members'),( 'one','security', 'software'),
                    ('one','high','level'),('one', 'field'),('one','coming'),('needs','plus', 'one'),( 'one', 'team'),('product', 'manager'),('day', 'one'),('take', 'risks'),('one', 'three',  'strategic',  'global'),('one','kind'),('3d','cameras'),
                    ('one','computer','science'),('one', 'front','end'),('one', 'best', 'software',  'products'),( 'one', 'dynamic', 'industries'),('great', 'people'),('one', 'deep','learning'),('one', 'international'),('experts', 'one'),
                    ('one', 'business'),('lead', 'one', 'growth'),('join','one','successful', 'leading'),('companies', 'echo', 'system'),('knowledge', 'one'),( 'one', 'startup'),('one','click'),('one', 'core'),('100', 'engineers'),('100', 'business'),
                    ( 'one',  'partners'),( 'one', 'leveraging'),( 'similar', 'one'),('delivery', 'one', 'hour'),('one', 'step','ahead'),('10', 'qa', 'engineers'),('100', 'customer'),('100',  'global', 'electrification'),('100','b'),('3d','bioprinting'),
                    ('500,000', 'companies'),('500,000', 'applications'),('10','technical','support'),('10','billion'),('2d','games'),('experience', 'team'),( 'good', 'knowledge'),( '3d','interactive'),( '3d',  'printed'),( '3d', 'virtual','events'),
                    ('animation', '3d'),( '3d', 'sensors', 'imagery'),('provider', '3d'),('fashion', 'design'),('3d', 'technology'),( '3d', 'stereo'),( '3d', 'modelling'),('macro', 'level'),('micro','controllers'), ('micro', 'fulfillment'), ('cro','specialist'),
                    ('3d','augmented','reality'),( 'unified','g'),('g','suite'),('okta', 'environments'),('dial','up'),('b2','g'),('l2','l3','protocols'),('micro', 'architecture'),('cro', 'similar'),('cro', 'tools'),('trend', 'micro', 'israel'),('millions', 'users'),('micro','external'),
                    ('angular','latest'),('headed','prof.'),('early','2018'),('approval', '2018'),('l3','l2', 'switching'),('teams', 'l2'),('micro', 'service'),('cro','provide'),('micro', 'frontends'),('low','latency'),('high','availability'),
                    ('low','amount'), ('low','software'), ('level','low', 'latency'),('low','variable'),( 'low','cost'),( 'low','risk'),('high', 'yield'),('low','latency'),( 'low', 'expenses'),('low', 'power'),( 'structural','validation'),
                    ('low','variable'),('low','latency'), ('data','processing'),('low', 'latency'),('low','response'),( 'low', 'latency'),('flow', 'design','system'),('follow','project'),( 'dvc','mlflow'),('least', '2'),('ron', 'johnson'),
                    ('sub','second'),('low','high','level'),('low','expenses'), ('fellow', 'team'),('flow','architect'),('hls', 'division'),( 'set','hls'),('security', 'hls'),('data','processes'),( 'pl', 'sql'),('one', 'main'),
                    ('anti', 'malware'),('av','algorithms'),('dr.' ,'yossi','kramer'),('av','industry'),( 'fw','av'),('anti', 'virus'),('digital', 'av'),('data', 'streams'),('av', 'automotive', 'industrial'),('comverse', 'tti'),  ('comverse', 'cisco'),
                    ('oo','design'),( 'design','patterns'),('native', 'oo'),( 'experience',  'oo'),('strong', 'oo'), ('design','programming'),('oo','modules'),('oo', 'language'),('oo','design'), ('oo', 'coder'),( 'oo',  'concepts'),
                    ('new', 'autonomous'),( 'av', 'application'),('yoav','levy', 'yonatan'),('two','seasoned'),('av','group'), ('av', 'anti', 'malware'), ('perception', 'av'),('av', 'nac'),('ct', 'scans'),('ct','security'),('ci','ct','cd'),
                    ('le','lte'),('transceiver', 'lte'),('lte','advanced'), ('lte', 'system'),('experience',  'test'),('dv', 'chip'),('implementation', 'hr'),('b','tests'),('b', 'test'),('gen','processes'),
                    ('hr', 'team'),( 'hr','tech'),('within', 'hr'),('hr','business'),('core', 'hr'),( 'hr','creation'),(  'hr', 'consulting'),('search','hr','consulting'),('hr', 'managers'),('helps', 'hr'),( 'learning','programs'),( 'hr', 'generalist'),('working', 'hr'),('part', 'hr'),(  'hr', 'related'),
                    ('must','ror','experience'),('framework', 'ror'),('web','technologies'),  ('component', 'vs'),('system','testing'),('gen','initiatives'),('asa', 'mcafee'),
                    ( 'drc','lvs'),( 'rfic', 'pa', 'lna'),( 'generation', 'circuitry'),('po', 'involves'),('service', 'po'),( 'xm', 'cybers'),('recruiters','od'), ('coverage', 'dimensions'),('nj', 'company'),
                    ('general','hr'),('policies','procedures'),('marketing', 'hr'),('engineering', 'teams'), ('hr', 'specialist'),( 'hr','programs'),( 'vp', 'hr'),('experience', 'hr'), ('creating', 'leading'),('financial', 'hr'),('functions', 'hr'),
                    ( '50', 'companies'),('50', 'worlds'),('help',  '50','people'),('50', 'hands'),('50,000', 'sqm'),('snort', 'bro'),('pe','degree'),('bd', 'team'),('bd', 'team'),('d.','experie'),('5','engineers'),
                    ('new', 'existing'),('new', 'applications'),('new','projects'),( 'new','techniques'),( 'new', 'hires'),( 'new', 'customer'),(  'new', 'technological'),('new', 'place'), ('new','unique'),( 'new', 'releases'),('new', 'systems'),
                    ('new','geographic'),('new', 'boundaries'),(  'new', 'leading'),( 'new', 'models'),('new',  'account'),( 'new','emerging'),('new', 'site'),(   'new','approaches'),( 'new', 'management'),( 'new', 'methodologies'),('new', 'tricks'),
                    ('new', 'cloud'),('new', 'test', 'cases'),( 'new','concepts'),( 'new','fields'),( 'new', 'plans'),('new', 'surgical'),('new', 'carrent', 'projects'),( 'new','enterprise'),('new','logics'),('new', 'tech', 'bottom'),(  'new', 'approach'),
                    ('new', 'state'),('new', 'thinking'),('new', 'exciting'),( 'new', 'content'),( 'new','idea'),('new','skills', 'technologies'),( 'new', 'domains'),( 'new', 'assessing'),( 'new','lines'),
                    ('new', 'problems'),( 'new','regional'),( 'new', 'versions'),( 'new', 'operating'),( 'new', 'ultra'),('new', 'generation'),('new', 'cd','pipeline'),( 'new', 'thing'),(   'new','expressive'),( 'new','economic'),
                    ( 'new', 'options'),('new','research'),(  'new', 'value'),('new','mobile'),( 'new', 'standards'),( 'new', 'relatives'),( 'new', 'ai'), ('new','deigns'),('new', 'technical'),( 'new', 'skills'),( 'new', 'ones', 'laugh'),
                    ('new','design','features'),( 'new','opening'),( 'new', 'ones', 'work'),( 'new','ones', 'support'),(   'new', 'infrastructure', 'team'),( 'new','databases'),(  'new', 'internal'),(  'new', 'uis', 'backed'),( 'new','clinical'),( 'new','proejcts'),
                    ('new','campaign'),( 'new','best', 'devops'),( 'new', 'mobility'),('new','possibilities'),(  'new', 'growth','opportunities'),(  'new', 'innovations','automation'),(  'new', 'breed'),('security','solutions'),
                    ('new','computers'), (  'new',  'framework'),( 'new', 'verticals'),( 'new','employees'),( 'new','tests'),( 'new', 'code', 'test'),( 'new','together', 'changed'),(   'new', 'dangerous'),( 'new', 'processes'),(  'new','multidisciplinary'),
                    ( 'new','variants'),( 'new','reseller'),('new','partnerships'),(  'new', 'networks'),( 'new', 'functionality'),('new', 'commercial', 'products'),( 'new', 'flow','continuous'),(  'new', 'members'),( 'new','collateral'), 
                    ( 'new', 'predictive'),(  'new', 'analog', 'circuit'),( 'new','initiative'),( 'new','environments'),('new', 'relationships'),( 'new','clients', 'territories'),( 'new', 'relationships', 'programmatic'),( 'new','jobs','teaching'),( 'new','prospects'),
                    ('new','core','product'),( 'new','rocketing', 'digital'),('new',  'image', 'modality'),( 'early', 'cancer'),(   'new','information'),( 'new', 'developed'),('new', 'information'),(  'new',  'monitors', 'automation'),( 'new', 'abilities', 'improve'),
                    ( 'experience','manage'),(   'new','components'),( 'new','house'),( 'new', 'firmware'),(  'new', 'soc'),('new', 'microservices'),(  'new', 'operational'),(  'new','initiatives'),(  'new','bpm'),('new','media'),('new', 'possibles'),
                    ('new', 'advanced'),( 'new', 'dates'),(  'new', 'additions'),( 'new', 'twist', 'classic'),(  'new','key'),('new','interns', 'support'),('transfer','knowledge'), ('learning', 'process'),( 'new', 'powerful',  'ways'),   
                    ('new', 'hyper', 'casual'),('new', 'growth', 'heights'),( 'new', 'major','development'),(  'new', 'era'),( 'new','log','source'),('new', 'imubit'),( 'new', 'kind','educational'),('experience',  'home',  'cooking'),
                    ( 'new', 'algorithms'),('new','video'),('related', 'features'),( 'new', 'video'),( 'new', 'website'),(  'new','role'),( 'new', 'modeling', 'analysis'),( 'new', 'problem'),( 'new', 'holistic'),('new', 'member'),('join','israeli'),('dev','department'),
                    ('new','associates'),( 'new', 'printing'),( 'new', 'associates'),(  'new', 'digital','department'),('new','car'),( 'new',  'professional', 'areas'),('new','hardware', 'software'),( 'new', 'world'),('least', 'year',  'fai'),('new','parts', 'projects'),
                    ('new', 'company'),( 'new', 'project'),( 'new',  'heuristics'),(  'new', 'strategic'),(  'new', 'frontiers'),( 'new', 'audio', 'text'),( 'new','raw'),(  'new','updated', 'features'),(  'new','publishers'),( 'new', 'strategies'),('new', 'delightful'),('new', 'datasets'),
                    ('new', 'developments'),('new', 'revolutionary'),( 'new', 'deployment'),( 'new','growth'),('new', 'development'),( 'new','titles'),( 'creating', 'new', 'nextgen'),( 'new', 'unity'),('new', 'head'),( 'new','service'),('new','sharing'),(  'new', 'procedures'),
                    ('new', 'process'),('new','quality','engineer'),( 'new','design'),( 'new','critical','suppliers'),( 'new','algorithmic','modules'),( 'new', 'threats'),('deep','web'), ('dark', 'web'),('new','reality','competing'), ('product', 'solution'),( 'new','sw', 'optimizations'),( 'new', 'organizational'),( 'new', 'uses'),
                    ( 'new','design'),('new','critical'),( 'adding', 'new', 'automated'), ('test', 'scenarios'),('new','intel'),( 'new', 'detection'),( 'new','activities'),( 'new', 'detection'),( 'new','vision'),('make', 'machine'),('data','accessible'),
                    ( 'included','ta'),('deep', 'knowledge'),( 'deep', 'tech', 'startup'),('deep', 'resources'),('deep', 'technology'),('deep','subject'),('deep','relevant'),('deep','networking'),( 'deep', 'familiarity'),('deep','dark','web'),('deep','dive', 'problem'),( 'deep','technical'), 
                    ( 'deep', 'dive','complex'),('deep', 'tech', 'technology'),('deep','tech','stack'),( 'deep','solutions'),('many','discussions'),('eep','knowledge'), ('deep','subject'),('deep','packet'),( 'deep', 'proven'),('deep','sense', 'ownership'),( 'deep','multi','disciplinary'),
                    ('deep','subject '),('deep','packet'),( 'deep', 'proven'),('deep', 'sense', 'ownership'),( 'deep','multi','disciplinary'),( 'deep','diving','complex'),('deep','background'),( 'deep','theoretical'),( 'deep', 'analysis'),('deep','user','research'),
                    ('deep','dive','various'),('deep','insights'),( 'deep', 'digital', 'investigation'),( 'deep',  'researching'),( 'deep', 'curiosity'),('deep','neural', 'network'),( 'new', 'innovation'), ( 'new', 'advancements'),( 'new','zealand'),( 'deep','diving','complex'),('deep','background'),( 'deep','theoretical'),
                    (  'deep', 'subject', 'matter'),( 'deep', 'dive', 'feature'),('deep', 'networks'),( 'deep', 'root'),( 'deep', 'debugging'),( 'deep', 'thinkers'),( 'deep','nets'),('deep', 'connections'),('deep','diving'),('deep', 'technological'),( 'deep', 'clinical'),('deep', 'innovation'),
                    ('deep','involvement'),('deep','analytical'),('deep', 'system'),(   'level', 'knowledge'),( 'deep', 'expertise'),( 'deep','rnns'),( 'deep','technically'),('deep','inspection'),('deep','security'),('deep','passion','startups'),
                    ('tech', 'marketing', 'data'),( 'deep','dive'),('deep', 'retail','industry'),('knowledge','experience'),('community','links'),( 'sar', 'technologies'),('experience','sw'),('sw', 'hw'),
                    ('deep','commitment'),( 'deep', 'ecosystem','industry'),( 'deep','troubleshooting'),('deep', 'os', 'stack'),('deep','dives'),('deep','rooted'),('qa','ra'),('ra','qa'),( 'deep', 'analysis'),('deep','user','research'),
                    ('deep','learning','tartup'),('houzz','shop'),('design','build','teams'),('sar','privately'),('held','boutique'),('focused','financial'),( 'lead','ra', 'team'),( 'sw', 'metrics'),(  'sw', 'expected'),('sw','skills'),( 'sw', 'systems'),
                    ( 'part','sw'),('techniques','sw'),('sw', 'engineering'),('closely', 'sw'),( 'leads','ar', 'platform'),( 'hw','rf', 'sw'),( 'hw','sw'),( 'sw', 'testing'),( 'sw','product'),( 'sw','development'),('sw','developments'),
                    ('pvt','monte'),('sw', 'engineer'),('sw', 'complex'),('sw', 'solution'),('sw', 'architecture'),('sw','implementation'),( 'sw', 'embedded'),('sw','stack'),
                    ('sw', 'mobile'),('sw','validation'),( 'sw', 'cloud'),( 'sw','design'),( 'sw',  'developers'),( 'sw', 'tools'),( 'sw', 'teams'),('sw', 'organization'),('degree',  'sw'),('enterprise', 'sw'),('rt','sw'),('sw', 'project','manager'),
                    ('sw','performance'),( 'sw', 'project'),( 'sw','components'), ('sw', 'requirements'),('sw','ee'),( 'sw', 'system'),( 'sw', 'architect'),( 'sw', 'fw'),('software','architect'),
                    ('teams', 'product'),('sw', 'groups'),( 'engineer', 'sw'),('experienced',  'sw'),('fundamental', 'sw','modules'),( 'interact','peers'),('sw','disciplines'),
                    ( 'hw', 'fw', 'sw'),( 'electronics', 'sw'),('data','plane','sw'),('sw', 'telecom'),(  'sw','products'),('manual', 'sw'),( 'complex', 'sw'),('design','data'),('data','analyst'),
                    ('sw', 'platform'),('teams','sw'),('trading', 'immediate'),( 'sw','division'),('senior','sw'),(  'rem','sw'),( 'similar', 'sw', 'work'),('sw','modules'),('data','center'),
                    ('embedded', 'sw'),('dev', 'organizations'),('cs', 'ce'),('ee','sw'),('bt','sw'),('model','selection', 'data'),( 'data','engineering'),('data', 'engineers'),('data', 'architecture'),
                    ('data','stores'),('data', 'storing'),('data','management'),('data','architecture'),('data','architectural'), ('knowledge', 'data'),
                    ('w', 'development'),('4','g'), ('1', 'million'),('n','1'),('based', 'w'),('least','b2', 'c1'),('sm','mm'), ('media', 'inc'),('channels',  'inc'),
                    ('di','frameworks'),( 'five', 'colleagues'),('five','consecutive'),('estate', 'five'),('five','continents'),('three','continents'),('five','studios'),('seven','countries'),
                    ('four', 'top'),('seven','retailers'),( 'seven','core'),('performance','seven', 'nines'),( 'seven', 'global', 'offices'),('least',  'two',  'years'),
                    ('product', 'data'),( 'data','model'),('high','data', 'integrity'),('data', 'related'),('data', 'related'),('data','visualization'),('statistics','science','data'),('systems', 'management'),
                    ('deep', 'thinking'),('deep','context'),('deep', 'rl', 'autonomous', 'machines'),('deep', 'hand'),('deep', 'hands'),( 'least', 'three', 'years'),('three', 'locations'),( 'three', 'reasons'),( 'three', 'dimensional'),
                    ('data', 'product'),('data', 'analytical'),('client','server'),('data', 'assets'),('data','daily'),('data','centers'),( 'bio', 'medical'),('bio', 'metrics'),
                    ('three','main', 'brands'),('including','three','former','founders'),( 'two', 'fold'),('every', 'single', 'day'),('three','renowned'),( 'three','global','financial'),
                    ('expert','one'),(   'one','downloaded'),('one','successful'),(  'one','best'),( 'hi', 'tech'),( 'optimization', 'one'),( 'one', 'messaging'),
                    ('least',  'two',  'years'),( 'two',  'days'),('least', 'two'),('two', 'brothers'),( 'two', 'decades'),( 'two','front'),('two','different', 'frameworks'),
                    ('two','three','technologies'),(  'two', 'following'),( 'two', 'subsidiary'),(  'two','founders'),('two', 'week'),('four', 'offices'),( 'past', 'four', 'decades'),
                    ( 'hundreds', 'millions',  'dollars'), ('per', 'year'),('100b','year'),( 'announced','year'),( 'lives', 'year'),( 'multi', 'year'),('brand','world','year'),
                    ('four','continents'),('eight', 'development'),('publication', 'year', 'award','app'), ('license', 'year'),   ('good', 'year'),( 'every', 'year'),
                    (  'many','years'),('recent', 'years'),('years','experience'),('10',  'years', 'experience'),( '7','years', 'experience'),('high', 'quality'),
                    ('node.js', '2'), ('ok', 'know'),('tens','thousands'),('tens','thousands'),('tens','millions', ),( 'tens','millions', 'sellers'),( 'tens','hundreds'),
                    ('billion', 'events', 'day'),('tens','billions'),('tens', 'gpus'),('20',  'engineers'),('israel', 'ltd.'),( 'group','ltd.'),( 'holdings', 'ltd.'),( 'instruments', 'ltd.'),
                    ('international', 'ltd.'),('tenengroup',  'ltd.'),( 'armor', 'ltd.'),('juniper','notebooks'),('juniper','notebook'),('high','attention'),('high','psychometric'),('high','throughput'),('high','paced'),
                    ('high','proficiency'),( 'high', 'budget'),( 'high', 'bmc'),('high','speed'),( 'high','accuracy'),( 'high', 'coding'),('high', 'standards'),(  'high','documentation'),('high','achiever'),( 'high','volumes', 'data'),
                    ('high','energy'),('high', 'velocity'),('high','standard'),('high','automation'),( 'times',  'day'),( 'high', 'impact'),( 'work', 'fast'),
                    ( 'high', 'network'),( 'high', 'visibility'),( 'high', 'initiative'),( 'high', 'education'),('high','familiarity'),('high','technical', 'skills'),( 'high', 'tolerance'),('high', 'volume'),
                    ('high', 'degree'),( 'high', 'dynamic'),('high', 'ability'),( 'high', 'knowledge'),('high','sense'),( 'high', 'read',  'write'),
                    ('high', 'presentation', 'skills'),( 'high', 'collaboration', 'skills'),('high','levels'),('high','intensity'),('high', 'technical'),
                    ( 'high', 'scalability'),('high', 'complexity'),( 'large', 'projects'),( 'high',  'traffic'),( 'high', 'priority'),('high','interpersonal', 'skills'), ( 'high', 'integration',  'skills'),
                    ('high', 'load'),('zero', 'downtime'),('high', 'professional', 'skills'),( 'high', 'goals'),( 'high', 'capacity'),('high', 'end','performance'),('high','motivation','sense'),( 'high', 'work'),
                    ('high',  'end','multi', 'core'), ('high', 'end',  'interactive'),( 'high', 'low','level'),( 'high', 'charts'),( 'high', 'eq', 'enjoy'),( 'high', 'pace'),( 'high', 'value'),
                    ('high','end', 'technologies'),( 'high',  'volumes'),('high','end', 'data','rate'),( 'high', 'data', 'rate'),('high', 'expectations'),( 'high', 'bw', 'circuits'),
                    ('high','data', 'rich'),(  'high',  'available'),( 'high','variance'),( 'high', 'frequency'),( 'high', 'motivation'),( 'high',  'frequency',  'trading'),('2','yearsofexperiencein'),
                    ( 'high','end','enterprise','security'),( 'high',  'desire'),('high', 'fidelity'),( 'high', 'analytical'),(  'high',  'resolution'),( 'high',  'professional'),(  'high', 'customers'),('high', 'english', 'level'),
                    ('high', 'involvement'),( 'high', 'scales'),( 'high', 'grade'),(  'high', 'interpersonal'),( 'high', 'production'),(  'high', 'ad', 'quality'),( 'high', 'reliability'),( 'high', 'end', 'saas'),
                    ( 'high', 'definition'),( 'high', 'image'),('high', 'service'),( 'high', 'cost', 'managing'),('high','end','consulting'),( 'high', 'grades'),('high', 'ambitions'),
                    ( 'high','security'),('high', 'score'),( 'high', 'school'),( 'high','granular'),( 'high',  'prioritization', 'skills'),('high', 'os', 'kernel'),( 'high', 'end',  'look'),( 'high', 'end', 'security'),
                    ( 'high', 'client'),( 'high','touch'),( 'high', 'frequencies'),(  'high', 'customer'),('high', 'precision'),('high', 'complex'),('project','management'),( 'high','stakes', 'performance'),('high', 'spirits', 'supporting'),
                    ('high','troubleshooting'),('high', 'end','leading'),( 'high', 'intellectual'),( 'high', 'business'),(  'high', 'efficiency'),('high', 'end', 'technology'),( 'high','efficiency','system'),('high', 'dau'),
                    ('high',  'technological'),('high','pressure'),('high', 'competence'),(  'high', 'standardized'),('high', 'power'),( 'high', 'currents'),
                    ( 'high', 'english'),( 'high',  'ethical',  'standards'),( 'high', 'interference'),( 'high',  'human','relations'),( 'work', 'alongside', 'customers'),( 'work', 'anywhere'),( 'work',  'dynamic'),(  'work','migrating'),
                    ('false', 'detections'),  ('work', 'together'),( 'work', 'environment'),( 'work',  'creative'),( 'work', 'partner'),( 'work', 'closely'),( 'work', 'departments'),
                    ( 'work', 'customers'),(   'population', 'work'),(   'relevant', 'work'),( 'work','effectively'),( 'work', 'independently'),( 'work', 'accommodate'),
                    ( 'person', 'work'),( 'work', 'industry'),('dynamic', 'work'),('companies', 'work'),(  'work','management'),( 'work','wide','variety'),( 'work', 'improve',  'product'),( 'work','collaboration'),
                    ( 'work','knowledge'),( 'work','agile', 'environment'),('work','collaborate'),( 'work','highly', 'experienced'),('work', 'beyondminds'),
                    ( 'work', 'learn', 'fun'),('work', 'monitor'),( 'work', 'development'),( 'work',  'team'),('work',  'engineers'),(  'work','correctly'),( 'work', 'pressure'),
                    ('work','prototypes'),( 'work',  'prototype'),(  'work', 'peer'),( 'work',  'believe'),('work', 'application'),('work', 'part'),('mutli', 'thread'),
                    ('work',  'different'),('work',  'experience'),('work',  'done'),( 'work','home'),('work',  'done'),( 'work','home'),('pre','post'),
                    ( 'growing', '3x', 'year'),( 'pre', 'phase'),('pre','production'),( 'pre', 'sales'),( 'pre', 'set', 'criteria'),(  'pre', 'sale'),( 'pre', 'release'),
                    ('pre', 'launch'),('pre', 'si', 'readiness'),('sse2','sse3','sse4'),('avx','avx2'),('avx512','avx512','vnni'),('cissp', 'ceh'),('ceh','cissp'),
                    ('aex','prx'),('xjse', 'prx'),('world','wide'),('plug','ins'),(  'ben', 'bogin'),( 'ben','gurion'),('specialist','sa'),('group', 'sven'),('sa','sysops'),
                    ('ww','ame'),('emerging', 'growth', 'incubation'),('systems',  'ers'),('intel','validation', 'engineering'),('architecture','graphics','software'),
                    ('15', 'employee'),('ro','il'),('features', 'k',  'product'),('k','core','services'),(  'k', 'techops',  'group'),( 'topologies',  'k'),
                    ( 'experience', 'true'),( 'true','tech'),( 'true','passion'),( 'true', 'essence'),(  'true', 'devops'),('true', 'desire'),( 'true',  'differentiation'),
                    ( 'true', 'continuous'),( 'true', 'reputation'),('true', 'value', 'properties'),( 'true', 'leader'),('web','apps'),('e', 'discovery'),
                    ( 'true', 'peoples'),('true', 'help'),( 'true', 'roots'),( 'true','personalization'),(  'true', 'impact'),( 'show', 'true','colors'),( 'true',  'advocate'),('true', 'accurate'),('true','data'),('true','foodie'),( 'email', 'true'),
                    ('mobile',   'ad'),('sb',   'etc'),('ab','tests'),('global',   'e'),('ab', 'testing'),('ab', 'inbev'),
                    ('conf','d'),( 'e', 'support'),( 'e', 'square'),( 'e', 'verilog'), ('fintech', 'e', 'payments'),('proprietary', 'e'),( 'la', 'maison'),
                    ('co','innovators'),('co','viewing'),('co', 'ordinating'),('co', 'host'),('co','develop'),('best', 'practices'),('best', 'practice'),('best', 'product'),('best', 'coding'),
                    ('best', 'quality'),('best', 'results'),('best','experience'),('best', 'suitable'),('best', 'class'),('best', 'platform'),('best', 'working'),('best', 'minds'),('best','production'),
                    ('non',   'dev'),('best','business'),('best','job'),('best','candidates'),('best','work'),('life','still'),('best','user'), ('part', 'job'),
                    ('super', 'b'),('patient','engagement','platform'),('pe','elf'),('software','computer'),
                     # Terms of years of experience
                    ('minimum','three'),('seven','years'),('year', 'two', 'experience'),( 'year', 'experience'),('2', '3','year'),('two',  'years'),('least','four','years'),( 'four','years'), ('least','ten', 'years'),( 'ten', 'years', 'later'),
                    ('3', 'years'),('2', 'years'),('1','years'),('4','years'), ('5', 'years'), ('6', 'years'), ('7', 'years'),('8', 'years'),('9','years'),('10', 'years'),('12', 'years'),
                    ('15', 'years'),('least','2','years'),('least','1','year'),('least','3','years'),('least','4','years'),('least','5','years'),('least','6','years'),('least','7','years'),
                    ('least','9','years'),('least','10','years'),('least','15','years'),('least','1.5','years'),('1','year'),('2', 'experience'),('primavera', 'p6'),('5','6','architecture'),
                    ('6','devops', 'engineers'),('4','6','professional','experience'),('team','6', 'engineers'),('3','product'),( 'one', '10','desirable', 'israeli'),('june','2015'),
                    ('1','2','years'),('1','3','years'),('1','5','years'),('2','3','years'),('2','4','years'),('2','5','years'),('3','4','years'),('3','5','years'),('3','6','years'),
                    ('4','5','years'),('4','6','years'),('5','7','years'),('5','8','years'),('6','7','years'),('2','10', 'years'),('least','4'),('4','year'),('25','years'),
                    ('3','years','minimum'),('4','years','minimum'),('5','years','minimum'),('0','1','years'),('0','2','years'),('4','knowledge'),('more','25','years'),
                    ('minimum','2','years'),('minimum','3','years'),('minimum','4','years'),('minimum','5','years'),('minimum','7','years'),('minimum','10','years'),('five','years'),
                    ('minimum','one','year'),('minimum','two','years'),('minimum','three','years'),('minimum','five','year'),('minimum','2','5','years'),('minimum','4','5','years'),
                    ('least','2','year'),('number', '1'),('startup','2020','israel'),('1','2','year'), ('4', '8','years'),('2','3','years'),( 'least','5',  'year'),
                    ('19', 'years'),('30','years'),('1,5','year'),('secure','one'), ('one', 'strongest'),( '3','8','years'),('34','years'),('3','year'),('5', 'year',  'experience'),('six','years'),('2', 'year'),(  'past', 'year'),( 'last','year'),    
                     # Terms of company names
                    ('Comm','it'),('sar','technologies','ltd.'),('viz.ai', 'inc'),('Code','Oasis'),('culture','trip'),('cyber','armor','ltd'),('k','health'),('x','sight' ),('3d','systems'),('action', 'item'),('dia','imaging', 'analysis'),
                    ('align', 'technology'),('amenity', 'analytics'),('amos','spacecom'),('anagal', 'next'),('anthem' ,'inc.'),('antelliq', 'innovation', 'center'), ('aqua','security'),
                    ('akamai', 'technologies'),('aran', 'lab'),('argus', 'cyber', 'security', 'ltd'), ('automat','it'),('avivit','dabush', 'hashama'),('avronim', 'ltd'),('axon','vision'),('beach','bum'),
                    ('b','net'),('britannica', 'knowledge', 'systems'),('brix', 'software','ltd'),('booking','com'),('blackswantechnologies','ai'),('cadence','design', 'systems'),('cato', 'networks'),
                    ('cgs', 'tower', 'networks', 'ltd'), ('check', 'point', 'software', 'technologies', 'ltd'),('check', 'point'),('chegg', 'lnc'),('cisco', 'systems'),('clear', 'gate' ),('clika', 'hr'),
                    ('cynet', 'security'),('darioHealth', 'corp'),('cnvrg','io'),('datagen', 'technologies'),('dataloop', 'ai'),('deep', 'instinct'),('d','fend', 'solutions'),('dgn', 'games'),
                    ('doit', 'international'),('deeponcoloy','ai'),('elbit', 'systems', 'ltd'),('extreme', 'group'),('ferro', 'corporation'),('final', 'israel', 'ltd'),('belong','Life'),
                    ('houndations', 'deep', 'learning', 'lab'),('gav', 'systems'),('general','motors'),('r','y','b','placement','agency'), ('viz','ai','inc.'),('airspan', 'networks'),
                    ('halo', 'digital'),('hcl', 'technologies'),('herolo','fullstack','frontend', 'development'),('high', 'tech', 'company'),('ht', 'bioimaging'), ('indoor', 'robotics'),
                    ('infinity','labs', 'r&d'),('intuition','robotics'),('ips','hi','tech', 'human', 'resources' ),('digital','talent','gency'),('just', 'eat','takeaway'),('ag','tech'),
                    ('kms','lighthouse'),('kramer', 'electronics'),('kornit', 'digital'),('kramer', 'electronics'),('kroll', 'consulting'),('leo','lane'),('lls','ltd'),('lsa','recruit','ltd'),
                    ('machon', 'noam'),('magic', 'leap'),('marketing', 'envy'),('mavrik', 'dental', 'systems', 'ltd'),('mavrik', 'dental', 'systems','ltd'),('microchip', 'technology', 'inc.'),('monday','com'),
                    ('mobile', 'group' ,'ltd'),('moon', 'active'),('nano', 'dimension'),('natural', 'intelligence'),('ncr','corporation'),('nova', 'measuring', 'instruments', 'ltd'),
                    ('nucleon', 'cyber'),('p','z','projects'),('palantir', 'technologies'),('palo', 'alto', 'networks'),('panda', 'trading', 'systems'),('shutterfly', 'inc.'),
                    ('perforce','software'),('perion','network'),('phibro', 'animal', 'health'),('ping', 'identity'),('prisma', 'photonics'),('project','verte'),('promo','com'),('q','core'),
                    ('qualcomm', 'israel', 'ltd'),('razor', 'labs'),('red','hat'),('redis', 'labs'),('regulus', 'cyber'),('s&p','global', 'ratings'),('sage','intacct'),('sage', 'intacct', 'inc.'),
                    ('sar', 'technologies', 'ltd'),('scopio', 'labs'),('seeking', 'alpha'),('shapeshift', 'gaming', 'ltd'),('shibumi','al'),('signals','analytics'),('next','gen'),
                    ('sodastream','international','ltd'),('solidus','tech'),('sophtix', 'security'),('special', 'job'),('sparks', 'ab'),('spiral', 'solutions'),('sqlink','group'),
                    ('sysaid','technologies', 'ltd'),('tailor', 'brands'),('tango', 'me'),('tenengroup', 'ltd'),('terasky','group'),('teva', 'pharmaceuticals'),('transmit', 'security'),
                    ('trax', 'retail'),('trend', 'micro', 'amea'),('upland', 'software'),('upstart', 'hr'),('upstream', 'security'),('valid', 'network'),('venus', 'concept'),('2017', 'proud'),('927','million'),
                    ('verint', 'cyber', 'intelligence', 'solutions'),('voyager', 'labs'),('waterfall', 'security'),('xsight', 'systems'),('sysaid', 'technologies', 'ltd'),('astronomical', 'it'),
                    ('ex', 'libris'),('verint', 'cyber', 'intelligence'),('eat', 'takeaway'),('applied', 'materials'),('confidential', 'company'),('takeaway','com'),('imdsoft' ,'ltd'),('snap','inc.'),
                    ('one', 'hour', 'translation'),('facto', 'media'),('nano', 'dimension'),('cross','discipline'),('final', 'israel','ltd'),('monday','com'),('globality', 'inc.'),('webpals', 'group'),
                    ('align', 'technology'),('zebra', 'medical', 'vision', 'ltd'),('dynamic', 'yield'),('rakuten', 'viber'),('rakuten', 'viber'),('stealth', 'mode', 'startup'),('gigaSpaces','technologies'),
                    ('blue', 'white', 'robotics'),('develo','source'),('k','health'),('n.', 'harris'),('forescout','technologies','inc.'),('technologies', 'ltd.'),( '888', 'group'),
                    ('cyber','security','ltd.'),('888','holdings','ltd'),('random', 'logic', 'ltd'),('corephotonics', 'ltd'),('health','k'),('fx', 'empire'),('viz.ai','inc.'),
                    ('solutions', 'inc.'),('tenable','inc.'),('globe',  'inc.'),( 'learn', 'ins',  'outs'),(  'check', 'ins'),('global',   'e',   'start'),
                    # Terms of Job scope and Location
                    ('full','time'),('part','time'),('tel', 'aviv'),('tel', 'aviv', 'yafo'),('tel', 'aviv', 'jaffa'),('ramat', 'gan'),('rishon','lezion'),('bene', 'beraq'),('new','york'),
                    ('new','york', 'city'),('guatemala', 'city'), ('ness','ziona'),('givat','ada'), ('or','yehuda'),('beer','sheva'),('kfar','saba'),('mishmar','haNegev'),('hod', 'hasharon'),
                    ('tel','hai'),('migdal', 'haemeq'), ('united', 'states'),('qiryat','gat'),('manchester','city'),('rishon', 'leziyyon'),('nes', 'ziyyona'),('rosh','haayin'),('zichron', 'yaacov'),
                    ('bat', 'yam'),('ness', 'ziona'),('san','francisco'),('san', 'jose'),( 'san', 'mateo'),('boston','university'),('kefar',  'sava'),('bat','yam'),('petah', 'tikva'),('ono','valley'))

In [55]:
# Merge of multi words
df_after_molti_words=create_multiple_words(tuples_molti_words_all_data,df_filtered_stop_words)
df_after_molti_words.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, manager, bi, team_leader, senior,...","[marketing, internet, bi, software_development]","[plarium, playtika, k_health]","[plarium, founded_2009, 2017_acquired_aristocr...","[goal, optimize, campaigns, effective, combina...","[1_year, digital_marketing, experience, user_a...","[full_time, full_time, full_time]","[herzliya, herzliya, tel_aviv_yafo]"
1,"[senior, software_engineer, social, media, acc...","[software_development, infrastructure, compute...","[k_health, liveperson, youappi]","[k_health, transforming, world, medicine, know...","[looking, senior, software_engineer, design, i...","[proven, software_engineering, experience, int...","[full_time, full_time, full_time]","[tel_aviv_yafo, raanana, raanana]"
2,"[full_stack, team_leader, senior, data_scienti...","[computer_software, data_science, machine_lear...","[youappi, soluto, playstudios]","[youappi, leading, performance, based, mobile,...","[youappi, looking, hands, fullstack, team_lead...","[least_two_years, experience, full_stack, team...","[full_time, full_time, full_time]","[raanana, tel_aviv, sarona, tel_aviv_yafo]"
3,"[senior, graphics, engine, engineer, embedded_...","[software_development, engineering, engineerin...","[playstudios, action_item, palantir_technologies]","[playstudios, develops, multiplayer, real_time...","[seeking, talented, motivated, experienced, se...","[minimum_7_years, relevant_work, experience, a...","[full_time, full_time, full_time]","[tel_aviv_yafo, tel_aviv, tel_aviv]"
4,"[forward, deployed, software_engineer, machine...","[engineering, information_technology, machine_...","[palantir_technologies, cnvrg.io, cnvrg.io]","[palantir, passionate, building, software, sol...","[work_customers, around, globe, gain, rare, in...","[strong, engineering, background, preferred, f...","[full_time, full_time, full_time]","[tel_aviv, tel_aviv, jerusalem]"


In [56]:
# The database contains expressions that have the same meaning. We will create a dictionary designed to switch between phrases
word_dic_all_data = {'tel_aviv_jaffa':'tel_aviv_yafo','yafo':'tel_aviv_yafo','tlv':'tel_aviv','nes_ziyyona':'ness_ziona','data_scientist':'data_scientists','product_analytics':'product_analyst','team_lead':'team_leader','test_plans':'testing_plan',
            'artificial_intelligence':'al','medical_device':'medical_devices','product_analyst':'product_analytics', 'machine_learning':'ml', 'uality_assurance':'qa','data_base':'db','database':'db','team_leads':'team_leader',
            'java_script':'js','multiple_sclerosis':'ms','elastic_block_store':'ebs','object_oriented':'oop','start_up':'start_ups','united_states':'us', 'booking_holdings':'bkng','core_2.x_3.x.':'core_2.x_3.x','jobs':'job',
            'production_engineering':'product_engineer', 'product_engineering':'product_engineer','quality_engineering':'qe','draftKings': 'dkng','dedicated_intelligence_research':'dir','sen.':'sen','complex_sw':'sw_complex',
            'certificate_analysis': 'coas','technical_data_sheets':'tds','software_development_lifecycle':'sdlc','program_management_office':'pmo','director_construction':'doc','e.':'e','ideas_z.':'ideas_from_a_to_z',
            'project_management_professional':'pmp','fab_construction_enterprise':'fce','computer_vision':'cv','facilities_contracts_manager':'fcm','manufacturing_product_engineering':'mpe','model':'models','underpinning':'underpin',
            'multi_tasking':'multi_task','team_player':'team_players', 'data_sets':'data_set','paid_campaigns':'ppc','mobility_service':'maas','end_end':'end_2_end','blue_white_robotics':'bwr','eep_knowledge':'deep_knowledge',
            'subject_matter_expert':'sme','program_management_professional':'pgmp','cyber_attacks':'cyber_attack','software_engineering':'software_engineer','israeli_security_agency':'isa','deep_hand':'deep_hands','verifies':'verification',
            'new_markets':'new_market','new_products':'new_product','taking':'take','powering':'power','playing':'players','os_x':'macos','c.':'c','r&d.':'r&d','from_a_z':'from_a_to_z','data_processes':'data_processing',
            'data_loss_prevention':'dlp','hybrid_environments':'hybrid_environment','building':'build', 'built':'build','worldwide':'world_wide','tier_1':'tier_one','fullstack':'full_stack','deep_technological':'deep_technology',
            'edge360':'edge 360','iso9001':'iso_9001','iso13485':'iso_13485','etc.':'etc','environment':'environments','evaluating':'evaluate','178':'do_178','160':'do_160','vs.':'vs','b2_g':'b2g','sw_developments':'sw_development',
            'gamer':'game','games':'game','working':'works','work':'works','way':'ways','understanding':'understand','test':'tests','fiverrs':'fiverr','unity3d.':'unity3d','health_k':'k_health','deep_technical':'deep_technology',
            'technological':'technologies','technology':'technologies','teams':'team','talented':'talents','talents':'talent','systems':'system','r&d.':'r&d','rs485':'rs_485','r.':'r','aidocs':'aidoc','ee_sw':'sw_ee','verifying':'verification',
            'successful':'success','students':'student','strategic':'strategy','solutions':'solution','setting':'set','services':'service','model':'models', 'iot.': 'iot','ag_operations_z':'ag_operations_a_z','verified':'verification',
            'respond':'response','required':'requirements', 'responsibilities':'responsible' ,'responsibility':'responsible','requirements':'requires','programs':'program','one_clouds':'one_cloud','motivatedandroid':'motivated_android',
            'problems':'problem', 'orientated': 'oriented','organization': 'organizations', 'organizational':'organizations','operating':'operation','3d.':'3d','i2c.':'i2c','headed_prof.':'headed_prof','developerto':'developer',
            'operations':'operation', 'optimize':'optimization','makes':'make','manage':'management','managements':'management', 'manager':'managers','technologies_ltd.':'technologies_ltd','log4j.':'log4j','properties':'property',
            'measure':'measurements','meeting':'meet', 'methodically':'methodical','machines':'machine','levels':'level','learning':'learn','globally':'global','big_advantage':'advantage', 'iac.': 'iac','engineer_sw':'sw_engineer', 
            'environments':'environment','engineer':'engineer','engineering':'engineer','effectively':'effective', 'effectiveness':'effective', 'over_3,000_employees':'more_3,000_employees','new_networks':'new_network',
            'efficiency':'effective','developers':'develop','developers':'develop','developing':'develop','deliver':'delivery','delivered':'delivery','r&d.':'r&d','html5':'html_5','unified_g':'unified_all_g','least_two_years':'2_years',
            'development':'develop','developed':'develop','differentiate':'different','practical':'practices','preventing':'prevent','familiarity':'familiar','rnd.':'rnd','vr':'virtual_reality','new_algorithmic':'new_algorithm',
            'continuously':'continuous','communicate':'communication','communications':'communication','communicative':'communication','designing':'design','b.a.':'b.a','vue_js':'vue.js','new_developments':'new_development',
            'companies':'company','companys':'company','consultants':'consultant','consulting':'consultant','containerized':'containerization','communication_service_providers':'csps','5g.':'5g','new_deign':'new_deigns',
            'collaborate':'collaborative','ability':'able','accessible':'access','accounts':'account','adoption':'adopt','written':'write','writing':'write','skill':'skills','ecma_script_6':'es6','programming_languages':'programming_language',
            '3d_2d':'3d_from_2d','2d_3d':'2d_and_3d','4_year':'4_years','4_6_professional_experience':'4_6_years','office365':'office_365','values':'value','ph.d.':'ph.d','phd.':'ph.d','b_test':'b_tests','sw_fw':'fw_sw',
            'founding':'founded','grows':'grow','optimally':'optimal','optimized':'optimize','homegrown':'home','house':'home','image': 'images','inversion_control':'ioc.','least_one':'1_year','qa_ra':'ra_qa','two_years':'2_years',
            'improve':'improvement','idea':'ideas','independently':'independence','frameworks':'framework', 'effectively':'efficient','evolution': 'evolves','soc.':'soc','convolutional_neural_networks':'cnns',
            'tcp_http_stack':'http_tcp_stack','http_tcp':'tcp_http','tcp_ip':'ip_tcp','lan_wan':'wan_lan','multi_tasking':'multi_task','electrical_engineering':'ee','os_x':'macos', 'wi_fi':'wifi','data_storing':'data_stores',
            'organizational_skills':'good_organizational_skills','social_skills':'good_social_skills','communication_skills':'good_communication_skills','z_responsibility':'a_z_responsibility',"modi'in":'modiin',
            'innovate': 'innovative','integrated':'integrate','internal':'internals','leaders':'leader','learner':'learn','networking':'network','888holdings':'888_holdings','owning_process_z':'owning_process_a_z',
            'help':'helps','creating':'create','deploy':'deployment','deployed':'deployment','experts':'expert','availability':'available','analyzing':'analyze','k8':'k8s','i.e':'i.e.','least_2':'2_years','best_practices':'best_practice',
            'provider':'providers','providing':'provides','publicly':'public','reliability':'reliable','produces':'product','preventing':'prevent','applications':'application','i.t.':'i.t','new_deigns':'new_deign',
            'architecting':'architect','architecture':'architect','consumers':'consumer','bachelors':'bachelor','experienced':'experience','environments':'environment','robot_service':'raas','micro_services':'micro_service',
            'according':'accordingly','allowing':'allows','advantage':'advanced','innovative':'innovation','includes':'include','goals':'goal','customers':'customer','saas.':'saas','six_years':'6_years','one_year':'1_year',
            '1_years':'1_year','least_1_years':'1_year','least_2_years':'2_years','least_3_years':'3_years','least_4_years':'4_years','least_5_years':'5_years','least_6_years':'6_years','least_7_years':'7_years',
            'least_9_years':'9_years','least_10_years':'10_years','least_15_years':'15_years','least_1.5_years':'1.5_years','3_years_minimum':'3_years','4_years_minimum':'4_years', 'least_5_year':'5_years','5_year_experience':'5_years',
            '5_years_minimum':'5_years','minimum_2_years':'2_years','minimum_3_years':'3_years','minimum_4_years':'4_years','minimum_5_years':'5_years','minimum_7_years':'7_years','minimum_10_years':'10_years','least_1_year':'1_year',
            'minimum_one_year':'1_year','minimum_two_year':'2_years','minimum_three_year':'3_year','minimum_five_year':'5_year','minimum_2_5_years':'2_5_years','minimum_4_5_years':'4_5_years', 'experience_one':'1_year',
            'least_4':'4_years','five_years':'5_years','least_2_year':'2_years','1_2_year':'1_2_years','1,5_year':'1.5_years','3_year':'3_years','c.':'c','iso26262':'iso_26262', 'new_proejcts':'new_proejct','data_architecture':'data_architectural',
            'evangelising':'evangelist','evangelizing':'evangelist','manufacturing':'manufacturer','manufacturers':'manufacturer','manufacture':'manufacturer','manufactured':'manufacturer','manufacturability':'manufacturer',
            'freeing':'free','frees':'free','correlating':'correlate','correlation':'correlate','correlation':'correlate','reading':'read','readable':'read','reader':'read','reads':'read','readers':'read',  
            'coding':'code','codes':'code','xcode':'code','codecs':'code','codefuel':'code','coders':'code','helps':'help','helped':'help','helping':'help','helpful':'help','profiling':'profile','profiles':'profile','profiled':'profile',
            'multi_threading':'multithreading','multithreaded':'multithreading','multithreading':'multithreading','multi_threaded':'multithreading','threads':'thread','threading':'thread','threadx':'thread','threaded':'thread',
            'developmental':'develop','developments':'develop','developerle':'develop','developer':'developer','develops':'develop','supplying':'supply','costings':'cost','costs':'cost','costing':'cost','costume':'costumer','costumers':'costumer',
            'evolving':'develop','empowering':'empowerment','empowers':'empowerment','empowered':'empowerment','empower':'empowerment','urge':'urgent','urgency':'urgent','interoperability':'interoperable',
            'simpler':'simple','simplest':'simple','simplistic':'simple','simplified':'simple','simplicity':'simple','simply':'simple','simplifies':'simple','simplees':'simple','simplifying':'simple','prefered':'preference',
            'measuring':'measurement','measures':'measurement','measurable':'measurement','measurability':'measurement','measured':'measurement','measurements':'measurement','measuring':'measurement','measures':'measure',
            'understating':'understand','understands':'understand','understandable':'understand','understood':'understand','elegantly':'elegant','remotely':'remote','decorating':'decorator','decorators':'decorator',
            'locks':'lock','unlocking':'unlocks','blockers':'block','blocking':'block','undertaking':'undertake','undertaken':'undertake','takes':'take','stakes':'stake','payouts':'pay','payable':'pay','paying':'pay','payoneer':'pay','payoneers':'pay',
            'remotely':'remote','decorating':'decorator','decorators':'decorator','cleaner':'clean','cleaning':'clean','cleaners':'clean','cleaner':'clean','cleanliness':'clean','fuels':'fuel','fuelled':'fuel','fueled':'fuel',
            'writers':'write','writer':'write','writers':'write','writer':'write','writes':'write','seeing':'see','sees':'see','seeking':'seek','seen':'see','vm':'vmware','incidents':'incident','matrixed':'matrix','matrices':'matrix',
            'bu':'boston_university','remotely':'remote','decorating':'decorator','decorators':'decorator','cleaner':'clean','cleaning':'clean','cleaners':'clean','cleaner':'clean','cleanliness':'clean','.management':'management',
            'assembling':'assemble','assembly':'assemble','assemble':'assemble','assembled':'assemble','assemblies':'assemble','assemblies':'assemble','assembly':'assemble','assembled':'assemble','voltage':'volte','voltmeters':'volte',
            'organizing':'organize','organizations':'organize','organized':'organize','respecting':'respect','respects':'respect','respectful':'respect','respected':'respect','respective':'respect','arts':'art','designs':'design',
            'facilitating':'facilitate','facilitated':'facilitate','facilitate':'facilitate','facilities':'facility','transferring':'transfer','transferred':'transfer','adjusting':'transfer','referencing':'reference','references':'reference',
            'revolutionising':'revolution','revolutionizing':'revolution','revolutionizes':'revolution','revolutionary':'revolution','revolutionized':'revolution','revolutionize':'revolution','meetings':'meet','meetups':'meet',
            'blogging':'blog','blogs':'blog','apis':'api','shaping':'shap','shapes':'shap','reshaping':'reshape','shapeshift':'reshape','using':'us','suspects':'suspected','reducing':'reduce','reduces':'reduce','reduction':'reduct','reduced':'reduc','mapreduce':'reduc',
            'simplee':'simple','outdoors':'outdoor','reliably':'reliable','simplee':'simple','outdoors':'outdoor','reliably':'reliable','interrupting':'interrupt','interruptions':'interrupt','interrupts':'interrupt','non_interrupted':'uninterrupted','interruption':'interrupt',
            'adapting':'adapt','adaptable':'adapt','adaptability':'adapte','adaptive':'adapt','adapted':'adapte','adaptation':'adapte','supporters':'support','supportive':'support','supporters':'support','supported':'support','supporting':'support',
            'researchresults':'research','researched':'research','researches':'research','researching':'research','caching':'cache','memcache':'memcached','conferencing':'conference','conferences':'conference',
            'adapts':'adapt','initiating':'initiate','initiation':'initiate','initiatives':'initiate','initiator':'initiate','initiative':'initiate','parameters':'parameter','hyperparameters':'hyperparameter','specializes':'specialist',
            'specialising':'specialist','specialistto':'specialist','specializing':'specialise','specialized':'specialise','specialised':'specialise','specialization':'specialise','specialists':'specialist','specialistto':'specialist',
            'salaries':'salary','conducting':'conduct','conducs':'conduct','conductor':'conduct','electrician':'electrical','watching':'watch','watchfuls':'watch','watchful':'watch','betting':'bet','bets':'bet','wants':'want','wanted':'want','wanting':'want',
            'surfaces':'surface','surfacing':'surface','implemention':'implement','implemented':'implement','implementable':'implement','implementing':'implement','implementations':'implement','scopes':'scope','footage':'extent','scoping':'scope',
            'extensive':'extend','extensions':'extend','extensibility':'extend','extensively':'extend','extends':'extend','extending':'extend','extendable':'extend','extended':'extend','external':'externally','extension':'extend',
            'influencer':'influence','influencing':'influence','influential':'influence','influencers':'influence','groupe':'group','groups':'group','groupings':'group','sight':'sights','advantages':'advantage','scop':'scope','microscope':'microscopes',
            'tagging':'tag','tags':'tag','stages':'stage','sightx':'x_sightx','spaces':'space','starting':'start','starts':'start','started':'start','startups':'startup','start_ups':'startup','startingwith':'start',
            'statements':'statement','briefs':'briefings','debriefings':'briefings','clustered':'cluster','clusters':'cluster','clustering':'cluster','finding':'find','finds':'find','findings':'find',
            'viewing':'view','viewer':'view','viewed':'view','views':'view','reviewing':'review','reviewer':'review','reviews':'review','interviews':'interview','interviewing':'interview','preserving':'preserve','preserved':'preserve','things':'thing',
            'passing':'pass','passionately':'passionate','concerns':'concern','concerning':'concern','draws':'draw','drawings':'draw','bidding':'bid','bids':'bid','protected':'protect','protecting':'protect','protection':'protect',
            'supervisory':'supervising','supervised':'supervising','supervision':'supervising','supervisor':'supervising','supervise':'supervising','supervising':'supervising','protects':'protect','protecting':'protect','protections':'protect',
            'productize':'product','products':'product','productivity':'product','productization':'product','productizes':'product','production':'product','checklists':'checklist','releases':'release','analysing':'analyse','analyses':'analyse',
            'discovering':'discover','discoverorg':'discover','discoveries':'discover','discovered':'discover','discovers':'discover','discovery':'discover','interconnects':'interconnect','interconnected':'interconnect',
            'listings':'list','lists':'list','listener':'listen','listening':'listen','alerting':'alert','alerted':'alert','alerts':'alert','alerting':'alert','synchronized':'synchronization','synchronous':'synchronization',
            'connected':'connect','connections':'connect','connects':'connect','connectors':'connect','connectivity':'connect','connected':'connect','connection':'connect','connecting':'connect','connections':'connect','frames':'frame','framed':'frame',
            'predictions':'predict','predictive':'predict','predictable':'predict','predictability':'predict','predicting':'predict','triple':'tripling','predicting':'predict','predicts':'predict','prediction':'predict','skill':'skills',
            'imagination':'imaging','imagined':'imaging','imagine':'imaging','imaginable':'imaging','imagery':'imaging','imagery':'imaging','ambitious':'ambition','ambitiously':'ambition','ambitions':'ambition','jsons':'json',
            'translating':'translate','translation':'translate','translational':'translate','translates':'translate','translations':'translate','tracking':'track','tracks':'track','tracked':'track','tracker':'track','signals':'signal','signed':'sign',
            'contributions':'contribution','contributing':'contribution','contributor':'contribution','contributes':'contribution','contributeto':'contribution','contributors':'contribution','contributed':'contribution','contribute':'contribution',
            'containers':'contain','container':'contain','contains':'contain','containerisation':'containerization','containerizing':'containerization','partitioning':'partitions','assigned':'assign','assignment':'assign','assignments':'assign','assigns':'assign',
            'designed':'design','redesigns':'design','designers':'design','wiring':'wire','wireframes':'wire','wired':'wire','wirelessly':'rewire','wireless':'rewire','loads':'load','loading':'load','loaded':'load','workloads':'workload','uploads':'upload',
            'houses':'house','inhouse':'house','leaderchampioning':'leaderchampioning','dw':'news','ts':'typescript','minimum_three':'3_years','seven_years':'7_years','demonstrably':'demonstrate','demonstrations':'demonstrate','demonstrated':'demonstrate',
            'least_four_years':'4_years','four_years':'4_years','least_ten_years':'10_years','ten_years_later':'10_years', 'year_experience':'1_year','yers':'year','manipulations':'manipulation','manipulate':'manipulation','manipulating':'manipulation','manipulated':'manipulation',
            'searching':'search','esearch':'search','researchops':'research','researchers':'research','research':'research','researcher':'research','outsourcing':'outsource','outsourced':'outsource','outsourcing':'outsource','authorized':'authorize','authorization':'authorize',
            'picking':'pick','picked':'pick','pickers':'pick','picker':'pick','picking':'pick','troubleshooting':'troubleshoot','troubleshoots':'troubleshoot','ingredients':'ingredient','industrializing':'industry','industrial':'industry','leaving':'leave',
            'scaling':'scalable','scaled':'scalable','scales':'scalable','scala':'scalable','scalability':'scalable','scaleup':'scale','escalation':'escalate','escalations':'escalate','escalated':'escalate','maximize':'maximum','maximizing':'maximum','socialize':'social',
            'hailos':'hailo','boomi':'booming','bringup':'booming','bringg':'bring','bringing':'bring','brings':'bring','brining':'bring','tools':'tool','tooling':'tool','toolssuch':'tool','toolsets':'toolset','toolchain':'toolset','toolchains':'toolset','toolkits':'toolset',
            'housing':'house','household':'house','warehousing':'warehouses','lighthouse':'clickhouse','publishing':'publish','published':'publish','publishers':'publish','publishes':'publish','productionize':'productionisation','productionize':'productionisation',
            'virtualized':'virtual','virtually':'virtual','virtualization':'virtual','parallelize':'parallel','parallelization':'parallel','analyzes':'analyzer','analyzed':'analyzer','citizenship':'citizen','citizens':'citizen',
            'wizer':'waze','wizers':'waze','runwize':'waze','bmcs':'bmc','monetization':'monetize','monetisation':'monetize','seven_years':'7_years','minimum_three':'3_years','2_yearsofexperiencein':'2_years',
            'optimizers':'optimize','optimisation':'optimize','optimistic':'optimize','optimum':'optimize', 'least_three_years':'3_years','year_two_experience':'1_2_years','mounted':'mounting','amounts':'amount','spanning':'span','spans':'span',
            'past_year':'1_year','last_year':'1_year', "'cause":'cause','10_years_experience':'10_years', '7_years_experience':'7_years','juniper_notebooks':'juniper_notebook','notebooks':'notebook',
            'compromising':'compromise','compromise':'compromise','uncompromising':'uncompromisingly','system_management':'systems_management','similar':'similarity','similars':'similarity','similarly':'similarity','similarweb':'similarwebs',
            'least_2_following_expertise':'2_years','b2c.':'b2c','ip':'ips','communicating':'communication','communicates':'communication','communicators':'communication','pipes':'pipe','relationships':'relationship','docs':'doc',
            'conveying':'convey','paving':'pave','looking':'look','looks':'look','looker':'look','benchmarking':'benchmark','benchmarks':'benchmark','captivating':'captivate','b.sc.':'b.sc','b.s.':'b.s',
            'architectures':'architect','architectural':'architect','architected':'architect','architecturally':'architect','architects':'architect','letters':'letter','demonstrating':'demonstrates','demonstrating':'demonstrates','demonstrable':'demonstrates',
            'vlookups':'vlookup','marketer':'market','marking':'mark','marker':'mark','marketo':'mark','shifting':'shift','shifts':'shift','hitting':'hit','deloittes':'deloitte','options':'option','optional':'option','vars':'var',
            'installing':'install','installers':'install','installation':'install','installations':'install','installer':'install','installment':'install','installs':'install','installed':'install',
            'vlookups':'vlookup','marketer':'market','marking':'mark','marker':'mark','marketo':'mark','shifting':'shift','shifts':'shift','performing':'performant','perform':'performant',
            'performance':'performant','performs':'performant','performed':'performant','interacts':'interaction','interactive':'interaction','andperformance':'performant','highlighting':'highlight','highlights':'highlight',
            'reworked':'reworks','plariums':'plarium','mixed':'mixe','mixer':'mixe','mixes':'mixe','downloaded':'downloads','downloading':'downloads','living':'lives','newsletters':'newsletter','delivering':'delivery','deliverables':'delivery',
            'deliverable':'delivery','delivers':'delivery','deliveries':'delivery','defending':'defend','defends':'defend','defender':'defend','inspecting':'inspection','inspections':'inspection','inspectors':'inspection',
            'screening':'screen','screens':'screen','participating':'participation','participate':'participation','participates':'participation','participants':'participation','participating':'participation','participant':'participation','participatory':'participation',
            'convergent':'convergence','converged':'convergence','acting':'activities','actively':'activities','acts':'activities','shadowing':'shadow','triggering':'triggers','triggered':'triggers','ranking':'ranks','ranked':'ranks',
            'representing':'represent','represented':'represent','caring':'cares','carefully':'cares','carried':'cares','varied':'varieties','variance':'varieties','variations':'variation','variants':'variation',
            'balancing':'balance','balancers':'balance','balances':'balance','balanced':'balance','executed':'execute','executes':'execute','showing':'show','shows':'show','shown':'show',
             'supplied':'supply','supplier':'supply','suppliers':'supply','supplies':'supply','branding':'brand','branded':'brand','brands':'brand','resolving':'resolve','resolved':'resolve','checked':'check','checks':'check',
            'executing':'execution','executed':'execution','executes':'execution','suggests':'suggest','suggesting':'suggest','suggestions':'suggest','deep_rooted':'deep_root','falls':'fall',
            'achieved':'achieve','achieving':'achieve','achiever':'achieve','achievement':'achieve','achievable':'achieve','situational':'situation','situations':'situation','kinds':'kind','marketing':'market',
            'marketed':'market','marketers':'market','marketplaces':'market','marketplace':'market','catches':'catch','biocatchs':'biocatch','soaring':'soar','struggling':'struggle','knowing':'know','known':'know','knows':'know',
            'knowledges':'knowledge','knowledgeable':'knowledge','shopper':'shop','shoppers':'shop','shop':'shop','calculating':'calculation','calculations':'calculation','calculators':'calculation','calculated':'calculation',
            'copies':'copy','stats':'statistics','stations':'station','static':'statistics','statistician':'statistics','firsts':'first','matching':'match','matches':'match','striking':'strike','roots':'root',
            'perimeters':'perimeter','updating':'update','updated':'update','updates':'update','filings':'filing','files':'file','makefiles':'makefile','filters':'filter','filtering':'filter','filters':'filter',
            'statistical':'statistics','statuses':'status','customized':'customization','customizations':'customization','customizable':'customization','stressing':'stress','distressed':'stress','skils':'skill',
            'switching':'switch','switches':'switch','approaching':'approach','approaches':'approach','new_approaches':'new_approache','gathering':'gather','gathered':'gather','gathers':'gather','players':'player',
            'serviced':'service','streamlines':'streamline','streamlining':'stream','streamed':'stream','streamlined':'stream','streams':'stream','impacting':'impact','impactful':'impact','impacts':'impact',
            'posting':'post','postgres':'postgre','posts':'post','postings':'posting','prospecting':'prospect','prospective':'prospect','prospects':'prospect','elevating':'elevate','elevates':'elevate','elevated':'elevate',
            'represents':'represent','representations':'represent','representation':'represent','representatives':'represent','representative':'represent','offices':'office','officers':'officer','official':'officials',
            'acceptance':'accept','acceptance':'accept','come':'comes','networks':'network','networker':'network','networked':'network','outcomes':'outcome','fulfilling':'fulfill','filled':'fulfill',
            'ninjas':'ninja','exactly':'exact','democratize':'democratise','democratizing':'democratise','microcontroller':'microcontrollers','coping':'copy','prides':'pride','angles':'angle','adopting':'adopt','adopted':'adopt',
            'environmental':'environment','environmentally':'environment','environments':'environment','pbs':'pb','governmental':'government','governments':'government','augments':'augmentation','augmented':'augmentation','augmenting':'augmentation','augmentations':'augmentation',
            '2_year':'2_years','thetarays':'thetaray','controlling':'control','controller':'control','controllers':'control','controls':'control','micro_controllers':'microcontrollers','remaining':'remain','remained':'remain','remains':'remain',
            'ending':'end','ends':'end','friends':'friend','friendly':'friend','recommenders':'render','digitizing':'digital','digitized':'digital','digitization':'digital','kinds':'kind','mimecasts':'mimecast', 'digestible':'digest','nurturing':'nurture','releasing':'release','released':'release','bookers':'booking','instructor':'instruction','instructional':'instruction','instructions':'instruction',
            'defining':'definition','definitely':'definition','definitions':'definition','defines':'definition','excels':'excel','excellence':'excel','excellent':'excel','carrying':'carry','vectors':'vector','payus':'pay','payments':'payment',
            'redefines':'redefine','redefined':'redefine','redefining':'redefine','bonding':'bonds','reimagining':'reimagine','cooking':'cook','cookies':'cook','2_experience':'2_years','3_experience':'3_years','experiencing':'experience','experiences':'experience','experiencein':'experience',
            'towers':'tower','laptops':'laptop','earned':'learn','learns':'learn','elearning':'learn','reimagining':'reimagine','shortening':'short','shorter':'short','shortly':'short','shorten':'short','largest':'large','largely':'large','larger':'larg',
            'compliances':'compliant','compliance':'compliant','non_compliance':'noncompliance','sunbits':'sunbit','specifying':'specify','establishing':'establish','established':'establish','establishment':'establish','establishes':'establish',
            'licensing':'license','licensed':'license','licenses':'license','obtaining':'obtain','obtained':'obtain','productizing':'productive','productionisation':'productive','productive':'productive','shopping':'productive',
            'happenings':'happen','happens':'happen','covering':'happen','happening':'happen','calls':'call','calling':'call','called':'call','caller':'call','servicing':'service','serviceability':'service','servicesm':'service',
            'dynamically':'dynamic','dynamics':'dynamic','radically':'dynamic','dynamics':'dynamic','specifically':'specific','specifications':'specific','specifics':'specific','pushing':'push','solving':'solve','solves':'solve','solver':'solve',
            'specificationsand':'specific','specification':'specific','analytically':'analytical','locally':'local','localized':'local','localization':'local','localised':'local','deadlines':'deadline',
            'fixed':'fix','fixes':'fix','fixing':'fix','meaning':'meaningful','forverymeaningful':'meaningful','gleaning':'glean','leaning':'lean','attending':'attend','microservices':'microservice','micro_service':'microservice','servicenow':'service',
            'attendance':'attend','attendee':'attend','messaging':'message','messages':'message','buying':'buy','buyers':'buy','buyer':'buy','trained':'training','exporting':'export','recommendations':'recommender',
            'perceiving':'perception','percepto':'perception','perceive':'perception','headquartered':'headquarters','removing':'removal','removable':'removal','remove':'removal','removed':'removal','documentationuser':'documentation',
            'funding':'fund','funded':'fund','funds':'fund','inspiring':'inspire','inspired':'inspire','inspires':'inspire','inspiration':'inspire','inspirational':'inspire','recommends':'recommender','sellers':'seller',
            'receiving':'receive','receivers':'receiver','received':'receive','receives':'receive','means':'mean','work_prototypes':'work_prototype','intel_validation_engineering':'ive','recommended':'recommender',
            'employers':'employ','employers':'employ','employed':'employ','employer':'employ','employing':'employ','employs':'employ','employment':'employ','recommending':'recommender','recommendation':'recommender','new_ways':'new_way',
            'employees':'employee','thethingsemployees':'employee','routing':'route','routers':'route','routinely':'routine','routines':'routine','worlds':'world','worldwide':'world_wide','charting':'chart','gateways':'gateway',
            'iterations':'iteration','answering':'answer','answers':'answer','new_features':'new_feature','humanities':'humanity','hazardous':'hazards','recommending':'recommender','chartering':'chart',
            'validating':'validation','validates':'validation','validations':'validation','validate':'validation','validated':'validation','validators':'validation','charts':'chart','borders':'border','varying':'vary',
            'featuring':'feature','features':'feature','featured':'feature','work_experience_3_years':'3_years', 'work_manage':'work_management','cissp_ceh':'ceh_cissp','orders':'order','ordering':'order','technologie':'technologies',
            'work_collaborate':'work_collaboration','work_fast':'work_fast_paced','completing':'complete','completer':'complete','scripting':'script','scripts':'script','5gs':'5_g','5g':'5_g','raised':'raise','cabling':'cable','cables':'cable',
            'descriptors':'descriptions','descriptive':'descriptions','spends':'spend','spending':'spend','riskier':'risk','risks':'risk','gaining':'gain','gained':'gain','brokerage':'broker','thinker':'think','thinking':'think',
            'logics':'logic','logical':'logic','pandologics':'pandologic','suitability':'suitable','probably':'probability','probabilistic':'probability','sketching':'sketch','brokers':'broker','reported':'report','reports':'report',
            'skil':'skills','upskilling':'skills','skilful':'skils','skilled':'skils','prompted':'prompts','storytelling':'storyteller','storytel':'storyteller','storyboarding':'storyboards','tfs':'tf','reporting':'reporte',
            'continued':'continue','continually':'continue','continues':'continue','continuum':'continue','continuity':'continue','continuous':'continue','continual':'continue','fitting':'fit','arises':'arise','nutrino':'nutrinos',
            'samples':'sample','ideally':'ideal','cybers':'cyber','cybereason':'cybereason','cybercriminals':'cybercrimes','architecture_graphics_software':'images','fitv':'fit','fitted':'fit','e':'ee','navinas':'navina',
            'asking':'ask','asks':'ask','asked':'ask','tasks':'task','tasked':'task','tasker':'task','agritasks':'agritask','multitasked':'multitask','multi_task':'multitask','multitasking':'multitask','ab':'ab_test','ab_testing':'ab_test',
            'vehicles':'vehicle','bscs':'bsc','bsc.':'bsc','subscribers':'subscription','subscribes':'subscription','disruptions':'disrupt','disrupts':'disrupt','relying':'rely','securely':'secure','secures':'secure','tests':'test',
            'cybersecurity':'cyber_security','cyberattacks':'cyber_attack','disrupts':'disrupt','disruptive':'disrupt','disrupting':'disrupt','disrupted':'disrupt','disruption':'disrupt','secures':'secure','fits':'fit',
            'mitigating':'mitigate','.mitigation':'mitigate','mitigation':'mitigate','mitigates':'mitigate','mitigated':'mitigate','emerging_growth_incubation':'egi','devices':'device','profitable':'profit', 'profitability':'profit',
            'based':'base','bases':'base','baselining':'base','develepor':'develope','developer':'develope','develope':'develope','sw_teams':'sw_team','sw_products':'sw_product','sw_engineering':'sw_engineer','profits':'profit',
            'sw_developer':'sw_development','smartling':'smart','smarter':'smart','correctly':'smart','smartest':'smart','smartnics':'smart','smartphones':'smartphone','flavored':'flavor','mscs':'msc','msc.':'msc','ways':'way','makers':'maker',
            'differently':'different','differentiation':'different','differentiated':'different','differential':'different','r&ds':'r&d','waiting':'wait','flavors':'flavor','shutterflys':'shutterfly_inc.','shutterfly':'shutterfly_inc.',
            'backends':'backend','trending':'trend','trends':'trend','cynet':'cynet','actors':'actor','factors':'factor','refactoring':'refactor','subcontractors':'subcontractor','prioritisation':'prioritise','prioritizing':'prioritise',
            'dedicated_intelligence_research':'dir','multiplier':'multiple','multiply':'multiple','multiplier':'multiple','stripes':'stripe','dr._noah_liel':'dr_noah_liel','viz.ai':'viz.ai_inc','prioritize':'prioritise','prioritized':'prioritise',
            'todays':'today','workflows':'workflow','benches':'bench','tier2':'tier_2','candidates':'candidate','contractors':'contractor','css3':'css_3','a.i.':'ai','viz.ai_inc.':'viz.ai_inc','founders':'founder','coloring':'color','colorful':'color',
             'aims':'aim','aiming':'aim','aimed':'aim','reclaims':'reclaim','infrastructures':'infrastructure','infras':'infrastructure',
             'claims':'claim','shells':'shell','classifiers':'classifier','classified':'classifier','derived':'derive','derives':'derive',
             'matters':'matter','earning':'earnin','cargo':'car','carriers':'carrier','directors':'director','channels':'channel','computerized':'computer','computers':'computer',
             'hosts':'host','hosted':'host','hostile':'host','barriers':'barrier','involvement':'involve','reposifys':'reposify',
             'chains':'chain','involvement':'involve','involves':'involve','involved':'involve','involving':'involve','triaging':'triage',
             'segmenting':'segmentation','segmented':'segmentation','segment':'segmentation','segments':'segmentation','segment':'segmentation',
             'shared':'shares','offers':'offer','offerings':'offer','offering':'offer','offers':'offer','paired':'pair','pairing':'pair',
             'repairs':'repair','paired':'pair','firewalls':'firewall','discounts':'discount','discounted':'discount','cultivated':'cultivate','cultivating':'cultivate',
             'altering':'alternative','alternatives':'alternative','states':'state','statement':'state','estate':'state','guardicores':'guardicore',
             'manning':'manner','manned':'manner','affiliates':'affiliate','affiliated':'affiliate','innovations':'innovation','comprehensively':'comprehensive','rated':'rate',
             'sparked':'spark','sparkling':'spark','affiliated':'affiliate','affiliates':'affiliate','relates':'relate','related':'relate',
             'analysed':'analyst','analysis':'analyst','analyse':'analyst','analyser':'analyst','analysts':'analyst','clients':'client',
             'angeles':'angel','programmers':'programmer','registered':'register','proofreading':'proof','restricted':'restrictions','storylines':'storyline',
             'guardicores':'guardicore','auctioning':'auctions','standings':'standard','standards':'standard','scores':'score','mounting':'mount',
             'dashboarding':'dashboard','dashboards':'dashboard','overcoming':'overcome','proofs':'proof','proofed':'proof','lieys':'lies','assuming':'assume','cisos':'ciso',
             'applieds':'applies','applied':'applies','standardization':'standard','camteks':'camtek','strengths':'strength','strengthen':'strength',
             'strengthening':'strength','minds':'mind','mindsets':'mindset','minded':'mind','careers':'career','cloudinarys':'cloudinary','phases':'phase',
             'appflyers':'appflyer','flying':'fly','shutterfly_inc.':'shutterfly_inc','relates':'relate','related':'relate','depending':'dependent','fraudsters':'fraudster',
             'requested':'request','requests':'request','requesting':'request','portions':'portion','enrolled':'enrollment','dependencies':'dependent','dependence':'dependent','dependency':'dependent',
             'uniting':'unit','units':'unit','communication':'community','communities':'community','communicated':'community','permissions':'permission','dbas':'dba','migrations':'migration',
             'diving':'dividend','individually':'individual','sourced':'source','sourcing':'source','learners':'learn','learned':'learn','learnin':'learn','codebases':'codebase',
             'complies':'complince','accomplishments':'accomplishment','resonates':'resonate','resonated':'resonate','portfolios':'portfolio','compilers':'compiler','modifying':'modify',
             'csps':'csp','isps':'isp','spss':'sps','dsps':'dsp','isps':'isp','iisp':'isp','imaging':'image', 'owning':'owner','owners':'owner','websites':'website','prepared':'prepare','prepares':'prepare','preparing':'prepare','clouds':'cloud',
             'universities':'university','mathematical':'mathematics','mathematicians':'mathematics','conventions':'conventional','conventional':'conventional','constrains':'constrain','constrained':'constrain','constraints':'constrain',
             'contracted':'contract','contractual':'contract','contracts':'contract','contracting':'contract','contractor':'contract','contracted':'contract','cicd':'ci_cd','dentists':'dentistry','clicks':'click','inventory':'inventor','inventors':'inventor',
             'boasting':'boast','boasts':'boast','followed':'follow','follower':'follow','follows':'follow','tomorrows':'tomorrow','trains':'train','training':'train','trainees':'train','tuned':'tun','tuning':'tun','couples':'couple','coupled':'couple',
             'desired':'desire','desirables':'desire','arises':'arise','anonymized':'anonymous','anonymizing':'anonymous','informative':'information','informatica':'information','informatics':'information','inventors':'inventor','inventories':'inventor','transforming':'transformation',
             'configurations':'configuration','upgrades':'upgrade','upgraded':'upgrade','upgrading':'upgrade','transformations':'transformation','transformed':'transformation','transform':'transformation','transformer':'transformation','transformational':'transformation',
             'transformers':'transformation','transformative':'transformation','istransforming':'transformation','transforms':'transformation','covers':'cover','covered':'cover','covere':'cover','coverity':'cover','uncovering':'uncover','packaged':'package','packages':'package',
             'embraced':'embrace','embracing':'embrace','embraces':'embrace','sharply':'sharp','sharpest':'sharp','nearly':'near','identification':'identify','confidential':'confident','confidently':'confident','confidentiality':'confident','packaging':'package',
             'implements':'implement','implementors':'implement','implementation':'implement','identities':'identify','identified':'identify','identifying':'identify','manages':'management','managerial':'management','managed':'management','manageability':'management','managers':'management',
             'manages':'management','runways':'runway','pathways':'pathway','easyways':'easyway','forwarders':'forward','forwarding':'forward','forwarder':'forward','detailed':'detail','details':'detail','detailing':'detail',
             'conceptualizing':'conceptualization','conceptualize':'conceptualization','perfectly':'perfect','perfecto':'perfect','perfection':'perfect','perfectos':'perfect','perfecting':'perfect','lets':'let','runs':'run','graded':'grade','grades':'grade',
             'undergraduate':'underground','complexities':'complete','complexities':'complete','complexity':'complete','completed':'complete','completely':'complete','completeness':'complete','billions':'billion','unlocked':'unlock','unlocks':'unlock',
             'lovers':'lover','unleashed':'unleash','unleashes':'unleash','unleashing':'unleash','pocing':'poc','pocs':'poc','controlups':'controlup','decoding':'decoder','considering':'consider','considerable':'consider','considerations':'consider','considers':'consider','consideration':'consider',
             'considering':'consider','considered':'consider','considerable':'consider','considerations':'consider','considers':'consider','lines':'line','impressive':'impress','impressively':'impress',
             'growing':'grow','growers':'grow','conversations':'conversation','conversational':'conversation','compressors':'compression','compressed':'compression','techniques':'technique','innovates':'innovator','exposes':'expose','exposed':'expose',
             'organisations':'organisation','organisational':'organisation','homes':'home','suites':'suite','suited':'suite','fronts':'front','frontal':'front','ontologies':'ontology','innovates':'innovator','innovating':'innovator','innovators':'innovator',
             'accelerator':'acceleration','accelerate':'acceleration','accelerates':'acceleration','accelerated':'acceleration','accelerating':'acceleration','accelerators':'acceleration','innovaton':'innovator',
             'orchestrates':'orchestrate','orchestrated':'orchestrate','proxies':'proxy','effects':'effect','effective':'effect','talks':'talk','talking':'talk',
             'negotiating':'negotiation','negotiations':'negotiation','negotiate':'negotiation','cells':'cell','opinions':'opinion','opinionated':'opinion',
             'hacking':'hacker','hackers':'hacker','hack':'hackers','insightful':'insight','insights':'insightful','staging':'stage','mapping':'map','maps':'map',
             'evolutions':'evolutionary','revolutionised':'revolution','strategists':'strategies','strategically':'strategies','strategize':'strategies','ms.':'ms.c',
             'new_strategies':'new_strategc','championing':'champion','types':'type','typed':'type','qualified':'qualify','qualifying':'qualify','qualifications':'qualification',
             'youappis':'youappi','delivering':'delivery','superfly':'super','motivate':'motivation','motivated':'motivation','tenengroup_ltd.':'tenengroup_ltd',
             'motivations':'motivation','motivating':'motivation','motivational':'motivation','motivator':'motivation','templates':'template',
             'motivated':'motivation','endeavors':'endeavor','sdks':'sdk','venues':'venue','banking':'bank','banks':'bank','corephotonics':'corephotonics_ltd',
             'breathing':'breath','breathe':'breath','breathes':'breathe','scientists':'scientist','united':'unit','unites':'unit','honors':'honor','drivers':'driver',
             'happiness':'happy','happier':'happiy','happen':'happy','uniquely':'unique','uniqueness':'unique','opportunities':'opportunity','vibes':'vibe','optics':'optic','optical':'optic',
             'agencies':'agency','agencies':'agency','policies':'policy','points':'point','touchpoints':'touchpoint','aligns':'alignment','aligner':'alignment',
             'hands':'hand','handles':'handle','summarise':'summary','summarize':'summary','summaries':'summary','treats':'treatment','webdriverio':'webdriver',
             'treating':'treatment','treatments':'treatment','treat':'treatment','gates':'gate','atalign':'alignment','atalign':'alignment','aligning':'alignment','aligned':'alignment','align':'alignment',
             'compiling':'compiler','compiled':'compiler','compilation':'compiler','compile':'compiler','conditions':'condition','conditional':'condition','converging':'convergence','formerly':'former',
             'administering':'administer','administrating':'administer','administrative':'administer','administrations':'administer','administration':'administer',
             'describing':'describe','described':'describe','scheduling':'scheduler','schedulers':'scheduler','schedule':'scheduler','schedules':'scheduler','scheduled':'scheduler',
             'responses':'response','responsive':'response','responsibly':'response','responsible':'response','generated':'generate','generates':'generate','assistants':'assistant',
             'provided':'provide','provides':'provide','providers':'provide','degrees':'degree','patient_engagement_platform':'pep','disclosures':'disclosure',
             'engineering':'engineer','engineered':'engineer','engineerings':'engineer','engineers':'engineer','qaengineerwith':'qaengineer','computer_software':'software_computer',
             'aspiring':'aspire','aspires':'aspire','aspiration':'aspire','stabilize':'stability','stabilization':'stability','damaging':'damage','corp.':'corp','corps':'corp',
             'incorporating':'corporation','corporates':'corporation','corporate':'corporation','incorporated':'incorporate','incorporating':'incorporate',
             'processors':'processor','upsells':'upsell','hives':'hive','vhive':'hive','creates':'creat','rpcs':'rpc','trusteers':'trusteer','entirely':'entire','circuits':'circuit','cancers':'cancer',
             'aspects':'aspect','ranges':'range','stranger':'strange','grids':'grid','rides':'ride','riding':'ride','rider':'ride','riders':'ride','bridgz':'bridge','bridging':'bridge','bridges':'bridge',
             'targets':'target','retargeting':'target','targeting':'target','targeted':'target','exposures':'exposure','expose':'exposure','exposing':'exposure',
             'practicality':'practice','practices':'practice','practicalities':'practice','practiceboth':'practice','practiced':'practice','practicing':'practice',
             'builders':'builder','manger':'mange','mangers':'mange','assisting':'assistant','assistive':'assistant','assists':'assistant','assistance':'assistant',
             'assisted':'assistant','promoting':'promote','promotes':'promote','promoting':'promote','disseminating':'dissemination',
             'memories':'memory','memories':'memory','properly':'proper','defeating':'defeat','partials':'partial','militaries':'military','challenging':'challenge',
             'guarding':'guard','guards':'guard','integrators':'integrator','visitors':'visitor','challenges':'challenge','challenged':'challenge','challenger':'challenge',
             'experimental':'experiment','experiments':'experiment','experimentation':'experiment','leveland':'level','clinicians':'clinician',
             'simulates':'simulate','simulated':'simulate',
             'pipelines':'pipeline','piping':'pipe','uncovers':'uncover','documents':'document','documented':'document','documenting':'document','documentation':'document',
             'existing':'exist','exists':'exist','reflects':'reflect','reflecting':'reflect','reflection':'reflect','reflectometry':'reflectomet','storages':'storage','closes':'close',
             'closely':'close','closer':'close','closed':'close','gamified':'game','gaming':'game','gamers':'game','evolves':'evolve','evolved':'evolve',
             'pulsed':'pulse','peoples':'people','flexibility':'flexible','flexibly':'flexible','cohorts':'cohort','ensuring':'ensure','ensures':'ensure','batteries':'battery','hypervisors':'hypervisor',
             'charges':'charge','charger':'charge','appliances':'appliance','combining':'combine','combinations':'combination','seizing':'seize','invites':'invite','invited':'invite','invitations':'invite',
             'graduates':'graduate',
            }




In [57]:
# Expressions of the same meaning were replaced by a predefined representative.
df_after_molti_words=exchanging_words_representative(word_dic_all_data,df_after_molti_words)
df_after_molti_words.head()

,0,1,2,3,4,5,6,7
0,"[mobile, ua, managers, bi, team_leader, senior...","[market, internet, bi, software_development]","[plarium, playtika, k_health]","[plarium, founded_2009, 2017_acquired_aristocr...","[goal, optimization, campaigns, effect, combin...","[1_year, digital_marketing, experience, user_a...","[full_time, full_time, full_time]","[herzliya, herzliya, tel_aviv_yafo]"
1,"[senior, software_engineer, social, media, acc...","[software_development, infrastructure, softwar...","[k_health, liveperson, youappi]","[k_health, transformation, world, medicine, kn...","[look, senior, software_engineer, design, impl...","[proven, software_engineer, experience, integr...","[full_time, full_time, full_time]","[tel_aviv_yafo, raanana, raanana]"
2,"[full_stack, team_leader, senior, data_scienti...","[software_computer, data_science, ml, nlp, sof...","[youappi, soluto, playstudios]","[youappi, leading, performant, base, mobile, a...","[youappi, look, hand, full_stack, team_leader,...","[2_years, experience, full_stack, team_leader,...","[full_time, full_time, full_time]","[raanana, tel_aviv, sarona, tel_aviv_yafo]"
3,"[senior, graphics, engine, engineer, embedded_...","[software_development, engineer, engineer, inf...","[playstudios, action_item, palantir_technologies]","[playstudios, develop, multiplayer, real_time,...","[seek, talents, motivation, experience, senior...","[7_years, relevant_work, experience, advanced,...","[full_time, full_time, full_time]","[tel_aviv_yafo, tel_aviv, tel_aviv]"
4,"[forward, deployment, software_engineer, ml, e...","[engineer, information_technology, ml, startup...","[palantir_technologies, cnvrg.io, cnvrg.io]","[palantir, passionate, build, software, solve,...","[work_customers, around, globe, gain, rare, in...","[strong, engineer, background, preferred, fiel...","[full_time, full_time, full_time]","[tel_aviv, tel_aviv, jerusalem]"


In [58]:
Collection_of_documents=df_after_molti_words.values.tolist()
# Replace the date column with the list
date_marge=[[datetime.strftime(date[0], '%d.%m.%Y'), datetime.strftime(date[len(zip_columns[10][0])-1], '%d.%m.%Y')] for date in zip_columns[10]]
date_marge=[[' '.join(d)] for d in date_marge]
# Add the date column to the data frame to add to the file
preprocessing_list=[Collection_of_documents[document] +[date_marge[document]] for document in range(len(Collection_of_documents))]

In [59]:
# Create a file of the results of the preprocessing process
pd.DataFrame(preprocessing_list).to_csv(url+'preprocessing.csv',index=False,header=False)